__[Open and try this file online (Colab, Pro / Extended RAM needed)](https://colab.research.google.com/github/djekra/wiktionary_wortschatz/blob/master/1_wiktionary_parsen/x34_d01_tag_und_member.ipynb)__

# tag und member
Verarbeitungsschritt Nr.3. Setzt das Verzeichnis data_00 voraus, in dem die vorherige Verarbeitungsstufe (d00_mwparserfromhell) die bisherigen Ergebnisse gespeichert hat. 
* speichert in data_01
* erstellt die Felder tag und member 
* integriert lexeme_manuell, named_entities und Zahlwörter
* Löschen laut löschliste
* behandelt Partizipien
* formt die grundlegende Datenstruktur (s.u.)
* testet Lexeme gemäß lex_test level 1
* Datei wiktionary ist danach überflüssig
* Dauer: Colab 37 min, Altrechner 26 min


### Doku
* __wiktionary_lemma__ enthält alle Grundformen. Ihr Identifikator ist die lemma_id.
* __wiktionary_merkmal__ enthält Merkmale, die einer Grundform zugeordnet werden. Manche Merkmale sind Lexeme, andere nicht. Merkmale eines Adjektivs sind z.B. seine Steigerungen (das sind Lexeme der Grundform) oder ein Gegenteilswort zur Grundform (das sind keine Lexeme der Grundform).
* __wiktionary_merkmal_text__ enthält ebenfalls Merkmale zu einer lemma_id. Allerdings keine einzelnen Wörter, sondern Textabsätze. Z..B. Definitionen oder Beispielsätze.

* __Abkürzungen__ stehen immer in wiktionary_merkmal (merkmal abk) und manchmal auch als eigenständiges Lemma in wiktionary_lemma.
  In wiktionary_lemma werden Abkürzungen durch das member 'Abkürzung' gekennzeichnet. 
  Das gibt die Information, dass es sich lohnt, in wiktionary_merkmal nachzuschlagen, um die Bedeutung einer Abkürzung zu erfahren.

* __Partizipien__ stehen in wiktionary_merkmal. Ihr lemma ist das Verb im Infinitiv (das ist oft falsch eingetragen). Ihr tag lautet 'VVPP ADJA' für Ptz2 bzw. 'VVFIN ADJA' für Ptz1, ihr member ist Ptz1 bzw. Ptz2.
  Zusätzlich sind alle Partizipien aber auch als Adjektiv als eigenständiges lemma registriert. Dort weist data_id auf das Verb zurück.

## Import

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['in_colab']     = False
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 17:24:48


In [2]:
# pandas
if environment['in_colab']:
    !pip install --upgrade pandas
import pandas as pd

In [3]:
# alle anderen
try:
    import pandasklar as pak, bpyth as bpy, bj_nlp

except ImportError as e:
    !pip install pandasklar bj_nlp
    import pandasklar as pak, bpyth as bpy, bj_nlp
    
grid       = pak.grid                     # dTale-Nutzung gemäß Config
from functools import partial
gridt = partial(grid, backend='dtale')    # dTale-Nutzung erzwungen

check_mask = pak.check_mask      

## Einstellungen

In [4]:
# Einstellungen

# Was debuggen?
suche_debug = ['Euro','€','tugendhaftestes','zu viele','zuviele', 'zu viel']  
suche_debug = ['notgetan']  

gründlich          = True    # einige Arbeitsschritte können weggelassen werden, wenn gründlich = False
speicher_freigeben = False    # sollen alte Ergebnisse gelöscht werden
version            = '_all'
#version           = '_100K'


# verbose
pak.Config.set('VERBOSE', True)

# Darstellung der Dataframes
pak.Config.set('GRID_BACKEND', '')   # standardmäßig kein dTale
pd.set_option('max_colwidth', 2000)  # aber dafür breiter

# Load
verzeichnis_load                      = 'data_00'
wiktionary_filename                   = verzeichnis_load + '/wiktionary'              + version + '.pickle'  
wiktionary_trash_filename             = verzeichnis_load + '/wiktionary_trash'        + version + '.pickle'
wiktionary_merkmal_filename           = verzeichnis_load + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename      = verzeichnis_load + '/wiktionary_merkmal_text' + version + '.pickle'  

# Steuertabellen, externe und manuelle Daten
löschliste_filename                   = 'Steuertabellen/löschliste.xlsx'     
lexeme_manuell_filename               = 'Steuertabellen/lexeme_manuell.xlsx'       # manuell eingepflegte Lexeme 
named_entities_filename               = 'Steuertabellen/named_entities_02.xlsx'    # Vornamen, Nachnamen, Orte
lex_test_filename                     = 'Steuertabellen/lex_test.xlsx'     

# Save
verzeichnis_save                      = 'data_01'
wiktionary_lemma_filename_save        = verzeichnis_save + '/wiktionary_lemma'        + version + '.pickle'  
wiktionary_nolemma_filename_save      = verzeichnis_save + '/wiktionary_nolemma'      + version + '.pickle'  
wiktionary_merkmal_filename_save      = verzeichnis_save + '/wiktionary_merkmal'      + version + '.pickle'  
wiktionary_merkmal_text_filename_save = verzeichnis_save + '/wiktionary_merkmal_text' + version + '.pickle'  

#
# Einige hardcodierte Tags finden sich unter wiktionary Einzelkorrekturen
# !!!

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

GRID_BACKEND = 
--> setting backend= as default for all pandasklar functions



## Vorbereiten

In [5]:
# Verzeichnisse erzeugen
os.makedirs(verzeichnis_load, exist_ok=True) 
os.makedirs(verzeichnis_save, exist_ok=True) 

In [6]:
# Steuertabellen und Python-Files von Github herunterladen
import os
if not os.path.exists('Steuertabellen'):
   !git clone https://github.com/djekra/wiktionary_wortschatz.git
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/Steuertabellen Steuertabellen
   !cp -r wiktionary_wortschatz/1_wiktionary_parsen/*.py .
#   !rm -r wiktionary_wortschatz

In [7]:
# Lade Ergebnisse der vorherigen Verarbeitungsstufe von Google Drive, falls nicht lokal vorhanden
# Dieses Verzeichnis wurde in der vorherigen Verarbeitungsstufe angelegt 
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av /content/drive/MyDrive/{verzeichnis_load} .

In [8]:
# Weitere Imports
from x32_Funktionen                    import *
from Steuertabellen.s01_Steuertabellen import *
from collections                       import defaultdict

## Daten laden

In [9]:
# wiktionary laden     
wiktionary               = pak.load_pickle(wiktionary_filename)
wiktionary               = pak.drop_cols(wiktionary,['wikitext','section_id'])

wiktionary_trash         = pak.load_pickle(wiktionary_trash_filename)
wiktionary_trash         = pak.drop_cols(wiktionary_trash,['msg','section_id','wikitext'])
wiktionary_trash         = pak.rename_col(wiktionary_trash,'error','msg')

wiktionary_merkmal       = pak.load_pickle(wiktionary_merkmal_filename)
wiktionary_merkmal       = pak.drop_cols(wiktionary_merkmal,['section_id'])

wiktionary_merkmal_trash = wiktionary_merkmal.head(0)

wiktionary_merkmal_text  = pak.load_pickle(wiktionary_merkmal_text_filename)

837593 rows loaded
4152 rows loaded
5171417 rows loaded
654677 rows loaded


In [10]:
# laden

# pak.nnan(wiktionary_merkmal_text)

In [11]:
# Keine NaN
assert not pak.any_nan(wiktionary, without=['syllables','rhymes','flexion','ipa'])

In [12]:
# Keine Leerzeichen in der ID
mask = wiktionary.section_id2.str.contains(' ')
check_mask(wiktionary,mask,0)

check_mask:  0 rows


In [13]:
pak.memory_consumption(locals())

,name,rtype,size
0,wiktionary_merkmal,"(DataFrame, Series)",2.1 GB
1,wiktionary,"(DataFrame, Series)",769.9 MB
2,wiktionary_merkmal_text,"(DataFrame, Series)",326.5 MB
3,wiktionary_trash,"(DataFrame, Series)",4.0 MB
4,mask,"(Series,)",1.6 MB
5,translate_tag,"(DataFrame, Series)",39.6 KB
6,translate_tagZ,"(DataFrame, Series)",21.6 KB
7,_i4,"(str,)",6.2 KB
8,Stop,"(type,)",1.0 KB
9,Markdown,"(type,)",1.0 KB


### wiktionary anschauen

In [14]:
# suche_debug 
mask = wiktionary.title.isin(suche_debug)
wiktionary[mask].sort_values(['section_id2',])

,page_id,page_part,section_id2,title,lemma,inflected,pos,rhymes,section_len,flexion,ipa,syllables,lang,lang_code,error
449830,672369,0,notgetan,notgetan,nottun,True,{'Adjektiv': ['Partizip II']},NaN,273,NaN,[ˈnoːtɡəˌtaːn],"[not, ge, tan]",Deutsch,de,


In [15]:
#pak.sample(wiktionary)

In [16]:
# suche_debug 
mask = wiktionary_trash.title.isin(suche_debug)
grid(wiktionary_trash[mask].sort_values(['section_id2',]))

No rows


In [17]:
# wiktionary_trash
a = pak.analyse_freqs(wiktionary_trash, 'msg')
a

,msg,msg_count,msg_percent,graph
0,title zu viele Wörter,3910,94.2,###############################################
1,title zu kurz,206,5.0,##
2,lemma zu kurz,26,0.6,
3,lang_code ist nan,10,0.2,


### wiktionary_merkmal anschauen

In [18]:
# suche_debug 
mask = wiktionary_merkmal.data.isin(suche_debug)
a = wiktionary_merkmal[mask].sort_values(['section_id2','merkmal'])
#a

In [19]:
pak.sample(wiktionary_merkmal)

,section_id2,merkmal,node_debug,node_kontext,data,num,meta
0,Hallo,übersicht,Deutsch Substantiv Übersicht,Genus,n,NaN,<NA>
446,April_1,istName,<NA>,<NA>,exists,NaN,<NA>
1031,Liebe_1,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural,—,NaN,<NA>
18515,Rauch,nurWortart,Wortart fehlt,1,Substantiv,NaN,<NA>
20173,Musik,übersicht,Deutsch Substantiv Übersicht,Genus,f,NaN,<NA>
81347,spulgen,übersicht,Deutsch Verb Übersicht,Präteritum_ich,,NaN,<NA>
175097,ein-,Präfix,<NA>,<NA>,exists,NaN,<NA>
230371,Joch,herkunft,,Ü,𒄿𒌑𒃷,[],
1308756,μm,up,,,Einheit,[1],
1314648,&c.,def,,,et cetera,[1],


In [20]:
pak.analyse_cols(wiktionary_merkmal)

,col_name,datatype_instance,datatype,datatype_short,datatype_suggest,is_numeric,is_string,is_hashable,nan_allowed,mem_usage,ntypes,nunique,nnan,ndups,n,vmin,vmean,vmedian,vmax,vsum
0,__index__,int64,np.int64,int64,np.int32,True,False,True,False,80.0 B,1,5171417,0,0,5171417,0.0,2585708.0,2585708.0,5171416.0,1.337177e+13
1,section_id2,str,pd.string,string,,False,True,True,True,836.0 B,1,836708,0,4334709,5171417,&c.,NaN,NaN,μm,NaN
2,merkmal,str,pd.string,string,,False,True,True,True,676.0 B,1,36,0,5171381,5171417,Präfix,NaN,NaN,übersicht,NaN
3,node_debug,str,pd.string,string,,False,True,True,True,676.0 B,1,31,11465,5159921,5171417,,NaN,NaN,Wortart fehlt,NaN
4,node_kontext,str,pd.string,string,,False,True,True,True,593.0 B,1,235,11465,5159717,5171417,,NaN,NaN,Ü,NaN
5,data,str,pd.string,string,,False,True,True,True,766.0 B,1,725566,0,4445851,5171417,,NaN,NaN,𒄿𒌑𒃷,NaN
6,num,list,object,object,,False,True,False,True,512.0 B,1,224,1133103,4038090,5171417,[],NaN,NaN,[1225],NaN
7,meta,str,pd.string,string,,False,True,True,True,544.0 B,1,18571,1133103,4019743,5171417,,NaN,NaN,παθοs,NaN


In [21]:
# Häufigkeit der Merkmale
a = pak.analyse_freqs(wiktionary_merkmal,['merkmal','data'])
gridt(a)

36 rows


In [22]:
# ursprüngliche_merkmale merken
ursprüngliche_merkmale = wiktionary_merkmal.merkmal.unique()

## Vorverarbeitung
* Feld num in String wandeln, um dups entfernen zu können
* Dups entfernen
* wiktionary: flexion in defaultdict wandeln

In [23]:
# num ist leere Liste 
mask = (wiktionary_merkmal.num.str.len() == 0)  
#wiktionary_merkmal[mask]
wiktionary_merkmal.loc[mask,'num'] = ''

In [24]:
# fillna
wiktionary_merkmal['num'] = wiktionary_merkmal.num.fillna('')

In [25]:
# num ist Liste mit einem Eintrag >> diesen verwenden
mask = (wiktionary_merkmal.num.str.len() == 1)
wiktionary_merkmal.loc[mask,'num'] = wiktionary_merkmal[mask].num.str[0]
#wiktionary_merkmal[mask]

In [26]:
?pak.list_to_string

Signature: pak.list_to_string(series, sep=',')
Docstring:
Converts a Series of lists of strings into a Series of strings.
* sep: The separator, default is ','
Example:
df = pak.people()
df['history2'] = pak.list_to_string(df.history)    
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/pandas.py
Type:      function


In [27]:
# Liste in String wandeln, damit wir Dups entfernen können
# wiktionary_merkmal: list_to_string
wiktionary_merkmal['num'] = pak.list_to_string(wiktionary_merkmal.num)

In [28]:
# Dups entfernen. 
# Die Duplikate entstehen z.B. dadurch, dass für manche Merkmale nur der Linktext verwendet wird. 
# Beispiel Wolf_1, Merkmal kombi: die Wolf _mündet_, in die Wolf _mündet_ führt zu einem Duplikat.
mask = wiktionary_merkmal.duplicated()
wiktionary_merkmal = pak.drop_rows(wiktionary_merkmal, mask)

Delete 1902922 rows from 5171417


In [29]:
# lemma_lower
wiktionary['lemma_lower'] = wiktionary.lemma.str.lower() 

In [30]:
# wiktionary: flexion in defaultdict wandeln
# 1 min
wiktionary = pak.cols_to_defaultdict( wiktionary, col_dict='flexion')

In [31]:
# change_datatype
wiktionary = pak.change_datatype(wiktionary, verbose=False)

## score

In [32]:
# Basis ist erstmal die Länge des Wiktionary-Eintrags
wiktionary['score_base'] = pak.scale( wiktionary.section_len, 'compare_median') + 0.001

In [33]:
# Lemmas sind besser
wiktionary['f0'] = 0.5
mask = (wiktionary.title == wiktionary.lemma)
wiktionary.loc[mask,'f0'] = 2.0

In [34]:
# Je mehr _ die ID enthält, desto schlechter
wiktionary['f1'] = 1 / (1 + wiktionary.section_id2.str.count('_'))  

In [35]:
# Je weiter unten auf der Seite, desto schlechter
wiktionary['f2'] = 1 / (1 + wiktionary.page_part) 

mask = wiktionary.page_part == 0
wiktionary.loc[mask,'f2'] += 1

In [36]:
# Merkmale zählen. Je mehr Merkmale, desto besser
df = pak.group_and_agg( wiktionary_merkmal,
                        col_origins = ['section_id2','data',],
                        col_funcs   = ['',           'size'],
                        col_names   = ['',           'm_count'] )
wiktionary = pak.update_col(wiktionary, df, on='section_id2',col='m_count')
wiktionary['f3'] = (wiktionary.m_count**0.5).fillna(0.1)

2431787 rows less, now 836708 rows
update_col: 836708 cells written into new column


In [37]:
# alles zusammenrechnen
wiktionary['score'] = wiktionary.score_base * wiktionary.f0 * wiktionary.f1 * wiktionary.f2 * wiktionary.f3

In [38]:
spalten = ['page_id','page_part','section_id2','title','lemma','section_len','score_base','f0','f1','f2','m_count','f3','score']
pak.sample(wiktionary[spalten])

Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


2023-03-05 17:27:55,624 - ERROR    - Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


2023-03-05 17:27:59,837 - ERROR    - Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


,page_id,page_part,section_id2,title,lemma,section_len,score_base,f0,f1,f2,m_count,f3,score
0,0,0,Hallo,Hallo,Hallo,1935,2.752282,2.0,1.0,2.0,10.0,3.162278,34.81392
27,50,0,ich,ich,ich,86973,111.775359,2.0,1.0,2.0,29.0,5.385165,2407.714918
135,217,0,Haus,Haus,Haus,70143,90.198436,2.0,1.0,2.0,1126.0,33.555923,12106.767249
8706,14581,0,jugendsprachl.,jugendsprachl.,jugendsprachl.,340,0.529926,2.0,1.0,2.0,2.0,1.414214,2.997712
9954,17156,0,JAG,JAG,JAG,344,0.53819,2.0,1.0,2.0,1.0,1.000000,2.15276
13086,21980,1,North_Carolina_1,North Carolina,North Carolina,1055,1.624077,2.0,0.333333,0.5,9.0,3.000000,1.624077
18971,47220,2,Riß_2,Riß,Riß,84,0.001,2.0,0.5,0.333333,NaN,0.100000,0.000033
28940,75293,0,enthält,enthält,enthalten,335,0.519595,0.5,1.0,2.0,1.0,1.000000,0.519595
95827,239700,0,μm,μm,μm,1007,1.562538,2.0,1.0,2.0,5.0,2.236068,13.975769
96368,240350,0,&c.,&c.,&c.,430,0.715876,2.0,1.0,2.0,2.0,1.414214,4.049606


In [39]:
# Zwischenwerte löschen
wiktionary = pak.drop_cols(wiktionary, ['section_len','score_base','f0','f1','f2','m_count','f3'])

In [40]:
# Häufigkeiten score
pak.analyse_freqs( (wiktionary.score/50).round(0) * 50 ).head(10)

,item,item_count,item_percent,graph
0,0.0,727445,86.8,###########################################
1,50.0,88335,10.5,#####
2,100.0,12370,1.5,
3,150.0,3912,0.5,
4,200.0,1909,0.2,
5,250.0,1083,0.1,
6,300.0,660,0.1,
7,350.0,430,0.1,
8,400.0,283,0.0,
9,450.0,191,0.0,


In [41]:
# Einträge mit dem höchsten Score
a = wiktionary.sort_values('score',ascending=False).head(10)
spalten = ['section_id2','title','lemma','pos','flexion','score']
a[spalten]

Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


2023-03-05 17:28:05,490 - ERROR    - Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


,section_id2,title,lemma,pos,flexion,score
135,Haus,Haus,Haus,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Haus'], 'Nominativ Plural': ['Häuser'], 'Genitiv Singular': ['Hauses'], 'Genitiv Plural': ['Häuser'], 'Dativ Singular': ['Haus'], 'Dativ Singular*': ['Hause'], 'Dativ Plural': ['Häusern'], 'Akkusativ Singular': ['Haus'], 'Akkusativ Plural': ['Häuser']}",12106.767249
6315,schwimmen,schwimmen,schwimmen,{'Verb': []},"{'Präsens_ich': ['schwimme'], 'Präsens_du': ['schwimmst'], 'Präsens_er, sie, es': ['schwimmt'], 'Präteritum_ich': ['schwamm'], 'Präteritum_ich*': ['schwomm'], 'Partizip II': ['geschwommen'], 'Konjunktiv II_ich': ['schwämme'], 'Konjunktiv II_ich*': ['schwömme'], 'Imperativ Singular': ['schwimm'], 'Imperativ Plural': ['schwimmt'], 'Hilfsverb': ['sein'], 'Hilfsverb*': ['haben']}",8328.625148
12146,Schnecke,Schnecke,Schnecke,{'Substantiv': []},"{'Genus': ['f'], 'Nominativ Singular': ['Schnecke'], 'Nominativ Plural': ['Schnecken'], 'Genitiv Singular': ['Schnecke'], 'Genitiv Plural': ['Schnecken'], 'Dativ Singular': ['Schnecke'], 'Dativ Plural': ['Schnecken'], 'Akkusativ Singular': ['Schnecke'], 'Akkusativ Plural': ['Schnecken']}",7218.973809
862,Katze,Katze,Katze,{'Substantiv': []},"{'Genus': ['f'], 'Nominativ Singular': ['Katze'], 'Nominativ Plural': ['Katzen'], 'Genitiv Singular': ['Katze'], 'Genitiv Plural': ['Katzen'], 'Dativ Singular': ['Katze'], 'Dativ Plural': ['Katzen'], 'Akkusativ Singular': ['Katze'], 'Akkusativ Plural': ['Katzen']}",6405.19632
128,Wasser,Wasser,Wasser,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Wasser'], 'Nominativ Plural 1': ['Wasser'], 'Nominativ Plural 2': ['Wässer'], 'Genitiv Singular': ['Wassers'], 'Genitiv Plural 1': ['Wasser'], 'Genitiv Plural 2': ['Wässer'], 'Dativ Singular': ['Wasser'], 'Dativ Plural 1': ['Wassern'], 'Dativ Plural 2': ['Wässern'], 'Akkusativ Singular': ['Wasser'], 'Akkusativ Plural 1': ['Wasser'], 'Akkusativ Plural 2': ['Wässer']}",6229.017
2735,Papagei,Papagei,Papagei,{'Substantiv': []},"{'Genus': ['m'], 'Nominativ Singular': ['Papagei'], 'Nominativ Plural': ['Papageien'], 'Genitiv Singular': ['Papageien'], 'Genitiv Plural': ['Papageien'], 'Dativ Singular': ['Papageien'], 'Dativ Plural': ['Papageien'], 'Akkusativ Singular': ['Papageien'], 'Akkusativ Plural': ['Papageien']}",5875.737306
466,Krankenhaus,Krankenhaus,Krankenhaus,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Krankenhaus'], 'Nominativ Plural': ['Krankenhäuser'], 'Genitiv Singular': ['Krankenhauses'], 'Genitiv Plural': ['Krankenhäuser'], 'Dativ Singular': ['Krankenhaus'], 'Dativ Singular*': ['Krankenhause'], 'Dativ Plural': ['Krankenhäusern'], 'Akkusativ Singular': ['Krankenhaus'], 'Akkusativ Plural': ['Krankenhäuser']}",5780.693006
6139,Esel,Esel,Esel,{'Substantiv': []},"{'Genus': ['m'], 'Nominativ Singular': ['Esel'], 'Nominativ Plural': ['Esel'], 'Genitiv Singular': ['Esels'], 'Genitiv Plural': ['Esel'], 'Dativ Singular': ['Esel'], 'Dativ Plural': ['Eseln'], 'Akkusativ Singular': ['Esel'], 'Akkusativ Plural': ['Esel']}",4922.37385
24531,schräg,schräg,schräg,{'Adjektiv': []},"{'Positiv': ['schräg'], 'Komparativ': ['schräger'], 'Superlativ': ['schrägsten']}",4879.18033
7045,Band,Band,Band,{'Substantiv': []},"{'Genus': ['n'], 'Nominativ Singular': ['Band'], 'Nominativ Plural 1': ['Bänder'], 'Nominativ Plural 2': ['Bande'], 'Genitiv Singular': ['Bandes'], 'Genitiv Singular*': ['Bands'], 'Genitiv Plural 1': ['Bänder'], 'Genitiv Plural 2': ['Bande'], 'Dativ Singular': ['Band'], 'Dativ Singular*': ['Bande'], 'Dativ Plural 1': ['Bändern'], 'Dativ Plural 2': ['Banden'], 'Akkusativ Singular': ['Band'], 'Akkusativ Plural 1': ['Bänder'], 'Akkusativ Plural 2': ['Bande']}",4616.801168


In [42]:
#grid(wiktionary, pak.sample)

## Steuertabellen laden

In [43]:
blab.help(plan_merkmal_erstellen)

<class 'IPython.core.display.Markdown'>


<span style="font-size:larger;">plan_merkmal_erstellen():</span>

* definiert in Steuertabellen/s01_Steuertabellen.py
* `merkmal`: Name des Merkmals
* `template`: Name des Mediawiki-Templates, das diese Information enthält
* `collect`: Extractionsmethode
   * L extrahiert nur Links 
   * T extrahiert komplette Text-Absätze
   * Ü extrahiert Übersetzungen
   * P extrahiert Template-Parameter (Substring reicht)
   * E checkt nur auf Existenz
* `plan`:
   * 1: die Merkmale, die in wiktionary_merkmal gespeichert werden
   * 2: die Merkmale, die in wiktionary_merkmal_text gespeichert werden   
   * 3: Reihenfolge und Liste aller Merkmale aus flexion / übersicht <br>
        sowie die Merkmale aus named_entities und lexeme_manuell
* `sort`: Irgendwas zum Sortieren der Merkmale
* `is_lex`: Ist das Merkmal ein Lexem?    

In [44]:
# plan_merkmal Steuertabelle
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#grid(plan_merkmal)
pak.sample(plan_merkmal)

,merkmal,template,collect,plan,sort,is_lex
0,noSteig,kSt.,E,1,,False
12,def,Bedeutungen,L,1,,False
17,klein,Verkleinerungsformen,L,1,,True
102,syllables,,,3,z1 syllables,False


In [45]:
# translate_tag: Wortarten >> tags

try:
    del translate_tag
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
#grid(translate_tag)
pak.sample(translate_tag)

,quelle,ziel,notiz
0,Konjugierte_Form,LEX_V,Keine Grundform
76,Präfix,TRUNC,Wortartenlisten
101,LEX_V LEX_V,LEX_V,Vorrang Durchlauf2
185,ADJ1 ADJ1,ADJ1,Doppelung1
212,Vergleichspartikel,,löschmich


In [46]:
# translate_tagZ: tag >> tagZ und tagZZ

try:
    del translate_tagZ
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
#grid(translate_tagZ)
pak.sample(translate_tagZ)

,tag,wortart_lang,tagZ,tagZZ,is_mutable,to_lemma
0,AFIX,Präfix,FIX,Z,False,W
21,VAFIN,Hilfsverb,VER_hilf,V,True,V
25,PPER,Personalpronomen,PRO,P,True,W
55,$(,Satzzeichen,SYM,Y,False,W
63,TODO,(todo!),X_todo,X,False,W


In [47]:
# lex_test, lemma_test
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=1)
#grid(lemma_test)
pak.sample(lemma_test)

185 rows loaded
Delete 89 rows from 185


,level,lemma,lemma_tag_soll,lemma_member_soll,check_tag
0,1,…,$(,,
1,1,Ackermann,NE,Person_nachname,
4,1,angucken,VVFIN,manuell,
16,1,Bussi,NN,,
82,1,zurecht,PTKVZ,,


## Löschliste
* Excel-Datei aufbereiten
* wiktionary belöschen
* wiktionary_merkmal belöschen
* Falls löschliste und lexeme_manuell in Konflikt stehen, siegt lexeme_manuell. Es wird erst gelöscht und dann manuell ergänzt.

In [48]:
# Laden
löschliste = pak.load_excel( löschliste_filename )

607 rows loaded


In [49]:
# Format prüfen
löschliste['check'] = löschliste.isnull().sum(axis=1)
mask = löschliste.check != 3
error = check_mask(löschliste,mask,0, stop=False)
grid(löschliste,mask, color='green')

check_mask:  0 rows
No rows, mask filters them all away


In [50]:
löschliste = pak.drop_cols(löschliste,'check')
bpy.raise_if(error)

In [51]:
# Einzelne löschlisten

mask = löschliste.section_id2.notnull()
löschliste_section_id2 = list(löschliste[mask].drop_duplicates().section_id2)

mask = löschliste.lemma.notnull()
löschliste_lemma = list(löschliste[mask].drop_duplicates().lemma)

mask = löschliste.lex.notnull()
löschliste_lex = list(löschliste[mask].drop_duplicates().lex)
löschliste_lex += löschliste_lemma

mask = löschliste.data.notnull()
löschliste_data = list(löschliste[mask].drop_duplicates().data)

In [52]:
löschliste

,lex,lemma,section_id2,data
0,g'rad,<NA>,<NA>,<NA>
1,hapax legomenon,<NA>,<NA>,<NA>
2,Hapaxlegomenon,<NA>,<NA>,<NA>
3,Jacob Ruff,<NA>,<NA>,<NA>
4,SARS CoV 2 AMVV,<NA>,<NA>,<NA>
...,...,...,...,...
602,<NA>,<NA>,<NA>,endbetonte Form
603,<NA>,<NA>,<NA>,Laut Online-Duden kann es auch ein weiblicher (Berg-)Steiger sein.
604,<NA>,<NA>,<NA>,Plural der maskulinen Form
605,<NA>,<NA>,<NA>,homm


In [53]:
# löschliste_lemma nach löschliste_section_id2 übertragen
mask1 =  wiktionary.title.isin(löschliste_lemma)   &   (wiktionary.title == wiktionary.lemma)
mask2 = ~wiktionary.section_id2.isin(löschliste_section_id2)
mask = mask1  &  mask2
#wiktionary[mask]
löschliste_section_id2 += list(wiktionary[mask].section_id2)

Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


2023-03-05 17:28:08,381 - ERROR    - Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


In [54]:
# wiktionary belöschen nach section_id2
mask = wiktionary.section_id2.isin(löschliste_section_id2)   
wiktionary, wiktionary_trash = pak.move_rows( df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg='löschliste_section_id2' )       # Löschen

Move 504 rows from 837593


In [55]:
# wiktionary belöschen nach lex
mask = wiktionary.title.isin(löschliste_lex)   
wiktionary, wiktionary_trash = pak.move_rows( df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg='löschliste_lex' )       # Löschen

Move 51 rows from 837089
Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


2023-03-05 17:28:10,306 - ERROR    - Exception occurred while processing request: 'NoneType' object is not iterable
Traceback (most recent call last):
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "/home/me/mambaforge/envs/python_06/lib/python3.9/site-packages/dtale/views.py", line 2480, in get_data
    curr_dtypes = [c["name"] for c in global_state.get_dtypes(data_id)]
TypeError: 'NoneType' object is not iterable


In [56]:
# wiktionary_merkmal belöschen nach section_id2
mask = wiktionary_merkmal.section_id2.isin(löschliste_section_id2) 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='löschliste_section_id2')

mask = wiktionary_merkmal_trash.msg == 'löschliste_section_id2'
grid(wiktionary_merkmal_trash[mask],pak.sample)

Move 11008 rows from 3268495
13 rows out of 11008


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg
446,April_1,istName,<NA>,<NA>,exists,,<NA>,löschliste_section_id2
5236,Mutter_2,übersicht,Deutsch Substantiv Übersicht,Nominativ Plural,—,,<NA>,löschliste_section_id2
5962,Boot_1,herkunft,,Ü,boot,,,löschliste_section_id2
81347,spulgen,übersicht,Deutsch Verb Übersicht,Präteritum_ich,,,<NA>,löschliste_section_id2
344673,ABl.EG,def,,,Amtsblatt der Europäischen Gemeinschaft,1,,löschliste_section_id2
479758,daselbst,abk,,,das,1,,löschliste_section_id2
592682,festkehren,übersicht,Deutsch Verb Übersicht,Präsens_ich,festkehre,,<NA>,löschliste_section_id2
670864,γ-Aminobuttersäure,übersicht,Deutsch Substantiv Übersicht,Genus,f,,<NA>,löschliste_section_id2
737003,fettarme_Milch,übersicht,Deutsch adjektivisch Übersicht,Nominativ Plural stark,——,,<NA>,löschliste_section_id2
1233898,geheien,def,,,werfen,5,alemannisch,löschliste_section_id2


In [57]:
# wiktionary_merkmal belöschen nach lex
mask = wiktionary_merkmal.data.isin(löschliste_lex) 
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='löschliste_lex')

mask = wiktionary_merkmal_trash.msg == 'löschliste_lex'
grid(wiktionary_merkmal_trash[mask], pak.sample)

Move 2882 rows from 3257487
13 rows out of 2882


,section_id2,merkmal,node_debug,node_kontext,data,num,meta,msg
1950,sein,alt,,,Sebastian Brant,,,löschliste_lex
49302,Geld,herkunft,,Ü,gelt,,,löschliste_lex
50817,Kreuzer,abk,,,Xer,2,,löschliste_lex
177755,Morphem,def,,,grammatisches Morphem,2,der,löschliste_lex
311374,fladern_1,übersicht,Deutsch Verb Übersicht,Partizip II,gefladerd,,<NA>,löschliste_lex
534518,absolut,kombi,,,absolute Dichtung,9,,löschliste_lex
633262,Piperidinsäure,def,,,γ-Aminobuttersäure,1,Chemie,löschliste_lex
1216474,-loge,def,,,gebundenes Lexem,1,,löschliste_lex
1432136,β-Ketobuttersäuren,gm,,,β-Ketobuttersäure,,,löschliste_lex
2196656,rußen,alt_1,,,russen,,Österreich,löschliste_lex


In [58]:
# aufräumen
del löschliste_lemma
del löschliste

## tag und member
* Aus dem Feld pos entstehen die Felder tag (standardisiert) und member (vollständig)

In [59]:
# Kaputte Datenstruktur in str wandeln
wiktionary['tag'] = wiktionary.pos.apply( lambda x: str(x) )
wiktionary = pak.move_cols(wiktionary,'tag','pos')

# translate
translate_chars = str.maketrans("{}'[]:", "     ,")
wiktionary.tag = wiktionary.tag.str.translate(translate_chars)

# Liste ohne Dups draus machen
wiktionary.tag = wiktionary.tag.str.split(",")
wiktionary.tag = wiktionary.tag.apply( lambda x: list(set(x)) )

# Strings pflegen
wiktionary.tag = wiktionary.tag.apply( lambda x: [ e.strip().replace(' ','_')  for e in x ] )

# Leere Strings entfernen
wiktionary.tag = wiktionary.tag.apply( lambda x: [ e for e in x  if e != ''] )

In [60]:
#grid(wiktionary, pak.sample)

In [61]:
# nach Priorität sortieren (laut Steuertabellen)
wiktionary.loc[:,'tag'] = member_sortieren(wiktionary.tag) 

In [62]:
# In string wandeln
wiktionary.tag = wiktionary.tag.str.join(' ')

In [63]:
# translate_first

print(translate_first1)
print(translate_first2)

wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_first1).str.strip()
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_first2).str.strip()

[('  ', ' '), ('Partizip II', 'Ptz2'), ('Partizip_II', 'Ptz2'), ('PartizipII', 'Ptz2'), ('Konjunktiv II', 'Konj2'), ('Konjunktiv_II', 'Konj2'), ('KonjunktivII', 'Konj2'), ('  ', ' ')]
[('  ', ' '), ('Partizip I', 'Ptz1'), ('Partizip_I', 'Ptz1'), ('PartizipI', 'Ptz1'), ('Konjunktiv I', 'Konj1'), ('Konjunktiv_I', 'Konj1'), ('KonjunktivI', 'Konj1'), ('  ', ' ')]


In [64]:
#mask = (wiktionary.tag.str.count(' ') + 1)  > 2   
#wiktionary[mask]

In [65]:
# Spalte member bewahrt die ursprüngliche Klassifizierung vollständig
wiktionary['member'] = wiktionary.tag.copy()
wiktionary = pak.move_cols(wiktionary,'member','tag')

In [66]:
# wiktionary anschauen
#mask = wiktionary.tag == 'ADJA ADV PRO'
mask = wiktionary.title.isin( suche_debug )
grid(wiktionary,mask)

1 rows out of 837038


,page_id,page_part,section_id2,title,lemma,inflected,pos,tag,member,rhymes,flexion,ipa,syllables,lang,lang_code,error,lemma_lower,score
449830,672369,0,notgetan,notgetan,nottun,True,{'Adjektiv': ['Partizip II']},Adjektiv Ptz2,Adjektiv Ptz2,NaN,{},[ˈnoːtɡəˌtaːn],"[not, ge, tan]",Deutsch,de,,nottun,0.391496


In [67]:
#pak.sample(wiktionary)

In [68]:
pak.sample(translate_tag)

,quelle,ziel,notiz
0,Konjugierte_Form,LEX_V,Keine Grundform
76,Präfix,TRUNC,Wortartenlisten
101,LEX_V LEX_V,LEX_V,Vorrang Durchlauf2
185,ADJ1 ADJ1,ADJ1,Doppelung1
212,Vergleichspartikel,,löschmich


In [ ]:
# anwenden
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_tag )
wiktionary['tag'] = wiktionary.tag.str.replace('  ',' ').str.strip()    

In [ ]:
# Durchlauf2: translate_tag erneut anwenden 
mask = translate_tag.notiz.str.contains('Durchlauf2')
wiktionary['tag'] = pak.replace_str( wiktionary.tag, translate_tag[mask] )
wiktionary['tag'] = wiktionary.tag.str.replace('  ',' ').str.strip()    

In [ ]:
# tag TODO für alles, was noch weiterverarbeitet werden muss
mask = (wiktionary.tag.str.len() == 0)
wiktionary.loc[mask,'tag'] = 'TODO'
#grid(wiktionary,mask)

In [ ]:
# wiktionary anschauen
#mask = wiktionary.tag == 'ADJA ADV PRO'
mask = wiktionary.title.isin( suche_debug )
grid(wiktionary,mask, backend='')

In [ ]:
# Dekliniertes_Gerundivum: beizubehaltende
# VMaßeinheiten

## Bindestrich

### Gebundene Adjektiv-Lexeme

In [ ]:
# Gebundene Adjektiv-Lexeme in wiktionary
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary.section_id2.isin(suche)

check_mask(wiktionary,mask,0,4)
wiktionary[mask]

In [ ]:
wiktionary.loc[mask,'title']       = wiktionary[mask].title.str.replace('-','')
wiktionary.loc[mask,'lemma']       = wiktionary[mask].lemma.str.replace('-','')
wiktionary.loc[mask,'lemma_lower'] = wiktionary[mask].lemma_lower.str.replace('-','')
wiktionary.loc[mask,'member']      = 'Adjektiv'
wiktionary.loc[mask,'tag']         = 'ADJA'
wiktionary.loc[mask,'pos']         = None # {'Adjektiv': []}
wiktionary.loc[mask,'section_id2'] = wiktionary[mask].section_id2.str.replace('-','_glex')
wiktionary[mask]

In [ ]:
#pak.analyse_datatypes(wiktionary)

In [ ]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal.section_id2.isin(suche)
check_mask(wiktionary_merkmal,mask,80)
wiktionary_merkmal.loc[mask,'section_id2'] = wiktionary_merkmal[mask].section_id2.str.replace('-','_glex')
#wiktionary_merkmal[mask]

In [ ]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal_text
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal_text.section_id2.isin(suche)
check_mask(wiktionary_merkmal_text,mask,46)
wiktionary_merkmal_text.loc[mask,'section_id2'] = wiktionary_merkmal_text[mask].section_id2.str.replace('-','_glex')
#wiktionary_merkmal_text[mask]

In [ ]:
# Gebundene Adjektiv-Lexeme in wiktionary
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary.lemma.isin(suche)
check_mask(wiktionary,mask,36)

wiktionary.loc[mask,'title']       = wiktionary[mask].title.str.replace('-','')
wiktionary.loc[mask,'lemma']       = wiktionary[mask].lemma.str.replace('-','')
wiktionary.loc[mask,'lemma_lower'] = wiktionary[mask].lemma_lower.str.replace('-','')
# wiktionary[mask]

In [ ]:
# Gebundene Adjektiv-Lexeme in wiktionary_merkmal
suche = ['ober-','unter-','äußer-','inner-',]
mask = wiktionary_merkmal.data.isin(suche)
check_mask(wiktionary_merkmal,mask,0,10)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.replace('-','')
wiktionary_merkmal[mask]

### Präfix

In [ ]:
# AFIX 551
mask0 = ~wiktionary.lemma.str.startswith('-')   
mask1 =  wiktionary.lemma.str.endswith('-')  
mask2 = ~wiktionary.lemma.str[0].str.isdigit()
mask3 =  wiktionary.lemma.str.len() > 1
mask4 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3  &  mask4 
pak.check_mask(wiktionary, mask, 551)

wiktionary.loc[mask,'tag']    = 'AFIX ' + wiktionary[mask].tag
wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

In [ ]:
# dar: Präfix ohne Bindestrich
mask1 = wiktionary.lemma == 'dar'
mask2 = wiktionary.member.str.contains('Präfix')
mask = mask1  &  mask2
pak.check_mask(wiktionary, mask, 1)

wiktionary.loc[mask,'tag']    = 'AFIX TRUNC'
wiktionary.loc[mask,'member'] = 'Präfix'
wiktionary[mask]

In [ ]:
# Kontrolle und Statistik AFIX
mask_AFIX = wiktionary.tag.str.contains('AFIX')
a = wiktionary[mask_AFIX].sort_values(['section_id2'])
mask = a.tag != 'AFIX TRUNC'
check_mask(a,0)

df = pak.analyse_freqs(a, ['member','lemma'])
df = pak.drop_cols(df,'lemma_count')
grid(df)

### Suffix

In [ ]:
# ZFIX 480
mask0 =  wiktionary.lemma.str.startswith('-')
mask1 = ~wiktionary.lemma.str.endswith('-')
mask2 = ~wiktionary.lemma.str[-1].str.isdigit()
mask3 =  wiktionary.lemma.str.len() > 1
mask4 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3  &  mask4 
pak.check_mask(wiktionary, mask, 480)

wiktionary.loc[mask, 'tag']   = 'ZFIX ' + wiktionary[mask].tag
wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

In [ ]:
# Kontrolle und Statistik ZFIX
mask_ZFIX = wiktionary.tag.str.contains('ZFIX')
a = wiktionary[mask_ZFIX].sort_values(['section_id2'])
mask = a.tag != 'ZFIX TRUNC'
check_mask(a,0)

df = pak.analyse_freqs(a, ['member','lemma'])
df = pak.drop_cols(df,'lemma_count')
grid(df)

### Sonstige

In [ ]:
# MFIX >> löschen
mask0 =  wiktionary.lemma.str.startswith('-')
mask1 =  wiktionary.lemma.str.endswith('-') 
mask2 =  wiktionary.lemma.str.len() > 2
mask3 = ~wiktionary.tag.str.contains(' ')   
mask = mask0  &  mask1  &  mask2  &  mask3
check_mask(wiktionary, mask, 4)

msg = 'MFIX'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

In [ ]:
# FIX_ZU_KURZ >> löschen
mask1A = wiktionary.tag.str.contains('AFIX')
mask1B = wiktionary.tag.str.contains('ZFIX')
mask2  = wiktionary.lemma.str.len() <= 2
mask = (mask1A | mask1B)  &  mask2
check_mask(wiktionary, mask, 9)

msg = 'FIX_ZU_KURZ'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

In [ ]:
# Zirkumfix >> löschen
maskA = wiktionary.lemma.str.contains('-…-')
maskB = wiktionary.member.str.contains('Zirkumfix')
mask = maskA | maskB
check_mask(wiktionary, mask, 3)

msg = 'Zirkumfix'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

In [ ]:
# dar ist ein Präfix ohne Bindestrich
mask = wiktionary.member.str.contains('Affix')
check_mask(wiktionary, mask, 1)

wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member, ['Affix'])
grid(wiktionary, mask)

In [ ]:
# Kontrolle und Statistik TRUNC
mask = wiktionary.tag.str.contains('TRUNC')
a = pak.analyse_freqs(wiktionary[mask], ['tag','member','lemma'])
assert a.shape[0] == 2
grid(a)

## Einzelkorrekturen

In [ ]:
# member Häufigkeitsanalyse
a = pak.analyse_freqs(wiktionary,['member','lemma'], splits=[' ',None])
a = pak.drop_cols(a,'lemma_count')
gridt(a)

In [ ]:
# Einzelkorrekturen (das sind section_id2)       
einzelkorrekturen = { 'viel'        : 'ADJA PIAT',  # war: ADJA ADV PIAT CARD
                      'zig'         : 'ADJA',
                      'bis'         : 'KOUS APPR',      
                      'derentwegen' : 'PDAT PRELAT',
                      'Rheinland'   : 'NE',          # war: NN       
}

In [ ]:
# anschauen (alle title, nicht nur die section_id2)
mask = wiktionary.title.isin(einzelkorrekturen.keys())
df = wiktionary[mask].sort_values('section_id2')
spalten = ['section_id2','title','lemma','tag','member','score']
grid(df[spalten])

In [ ]:
# ändern lt. section_id2
for i,t in einzelkorrekturen.items():
    mask = (wiktionary.section_id2 == i)
    wiktionary.loc[mask,'tag'] = t

In [ ]:
# pflege_tag_felder (schreibe tagZZ usw.)
wiktionary = pflege_tag_felder(wiktionary, translate_tagZ)

In [ ]:
# anschauen (alle title, nicht nur die section_id2)
mask = wiktionary.title.isin(einzelkorrekturen.keys())
df = wiktionary[mask].sort_values('section_id2')
spalten = ['section_id2','title','lemma','tag','tagZZ','member','score']
grid(df[spalten])

## named_entities

In [ ]:
# laden
named_entities = pak.load_excel(named_entities_filename, tabcol='quelle')

In [ ]:
# Überblick
a = pak.analyse_freqs(named_entities, cols=['quelle','member','lemma'], splits=[None,' '])
a = pak.drop_cols(a,'lemma_count')
grid(a)

In [ ]:
# score anpassen
# es gilt nahezu: wiktionary.score.quantile(0.5)  == named_entities.score.quantile(0.5) 
named_entities.score *= wiktionary.score.quantile(0.5) / named_entities.score.quantile(0.5) 

In [ ]:
# je mehr Funktionen, desto besser
named_entities.score *= (1 + named_entities.member.str.count(' '))  

In [ ]:
pak.sample(named_entities)

In [ ]:
# Felder ergänzen
named_entities['title']       = named_entities.lemma
named_entities['lemma_lower'] = named_entities.lemma.str.lower() 
named_entities = pak.move_cols( named_entities, 'lemma_lower','lemma')

In [ ]:
# section_id2
named_entities['section_id2'] = named_entities.lemma + '_' + named_entities.quelle.str[0].str.upper().copy()
named_entities['section_id2'] = named_entities.section_id2.str.replace(' ','_').copy()
named_entities = pak.move_cols(named_entities, ['section_id2','title'])

In [ ]:
# Bekannte section_id2 aus wiktionary übertragen. Aber nur wenn der tag wirklich gleich ist.
mask = (wiktionary.lemma == wiktionary.title)  # wichtig
named_entities = pak.update_col(named_entities, wiktionary[mask], on=['lemma_lower','tag'], col='section_id2',  col_score='score' )

In [ ]:
pak.sample(named_entities)

In [ ]:
# drop_cols
named_entities = pak.drop_cols(named_entities,['sex_f', 'sex_m', 'Land', 'Stadt','quelle'])

In [ ]:
grid(named_entities, pak.sample)

In [ ]:
# Dups?
mask = named_entities.section_id2.duplicated()
check_mask(named_entities,mask,0,0)

In [ ]:
blab.help(pak.cols_to_dict)

In [ ]:
named_entities = pak.cols_to_dict(named_entities,col_dict='flexion',cols_add=['Genus','Nom Sg','Gen Sg',], use_defaultdict=True)
named_entities

In [ ]:
grid(named_entities, pak.sample)

In [ ]:
# anfügen
wiktionary = pak.add_rows(wiktionary, named_entities, assert_subset=True)

In [ ]:
# flexion type korrekt?
assert pak.ntypes(wiktionary.flexion) == 1

In [ ]:
# wiktionary anschauen
mask = wiktionary.title.isin(suche_debug) 
r = wiktionary[mask].sort_values(['title','section_id2'])
grid(r)

## Zahlwörter
* werden anschließend lexeme_manuell zugeschlagen

In [ ]:
# Wandelt eine Zahl in Worte um
def zahl_in_worten(n, s=True, z=False):
    
    w1 = "null ein zwei drei vier fünf sechs sieben acht neun zehn elf zwölf dreizehn vierzehn fünfzehn sechzehn siebzehn achtzehn neunzehn".split()
    w2 = "zwanzig dreißig vierzig fünfzig sechzig siebzig achtzig neunzig".split()    
    if n < 0: raise ValueError
    if n == 0 and z: return ""
    if n == 1 and s: return "eins"
    if n < 20: return w1[n]
    if n < 100:
        w = w2[(n - 20) // 10]
        if n % 10:
            w = w1[n % 10] + "und" + w
        return w
    if n < 1000:
        return w1[n // 100] + "hundert" + zahl_in_worten(n % 100, z=True)
    if n < 1000000:
        return zahl_in_worten(n // 1000, s=False) + "tausend" + zahl_in_worten(n % 1000, z=True)
    raise ValueError

In [ ]:
print(zahl_in_worten(3107))

In [ ]:
def zahlwörter_liste(range_): 
    return [ 
    {'lex' : [zahl_in_worten(z), str(z)], 
     'lemma' : zahl_in_worten(z),      
     'def'   : str(z),
    } for z in range_]

zahlwörter_liste(range(0,20,10))

In [ ]:
# 0..99 zahlwörter_a erzeugen
range_ = range(0,100)
zahlwörter_a = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_a['score'] = wiktionary.score.quantile(0.8)  +  1/(zahlwörter_a.index.astype(int)+1)**0.5*3+1
#zahlwörter_a
# wiktionary.score.quantile(0.7) / 

In [ ]:
# Hunderter Zahlwörter erzeugen
range_ = range(100,1000,100)
zahlwörter_b = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_b['score'] = wiktionary.score.quantile(0.8)  +  1/(zahlwörter_b.index.astype(int)+1)**0.5*3+1
#zahlwörter_b

In [ ]:
# Tausender Zahlwörter erzeugen
range_ = range(1000,99001,1000)
zahlwörter_c = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_c['score'] = wiktionary.score.quantile(0.6)  +  1/(zahlwörter_c.index.astype(int)+1)**0.5*3+1
#zahlwörter_c

In [ ]:
# Zehntausender Zahlwörter erzeugen
range_ = range(10000,99001,10000)
zahlwörter_d = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_d['score'] = wiktionary.score.quantile(0.5)  +  1/(zahlwörter_d.index.astype(int)+1)**0.5*3+1
#zahlwörter_d

In [ ]:
# Hunderttausender Zahlwörter erzeugen
range_ = range(100000,990001,100000)
zahlwörter_e = pd.DataFrame(zahlwörter_liste( range_ ))
zahlwörter_e['score'] = wiktionary.score.quantile(0.7)  +  1/(zahlwörter_e.index.astype(int)+1)**0.5*3+1
#zahlwörter_e

In [ ]:
# concat
zahlwörter_all = pd.concat([  zahlwörter_a, zahlwörter_b, zahlwörter_c, zahlwörter_d, zahlwörter_e,  ])

In [ ]:
# explode lex
zahlwörter_all = zahlwörter_all.explode('lex')

In [ ]:
# Sonstige Spalten
zahlwörter_all['tag']         = 'CARD'
zahlwörter_all['tag_lex']     = 'lex' # ja
zahlwörter_all['genus']       = 'f'
zahlwörter_all['member']      = 'Zahlwort'
zahlwörter_all['tab']         = 'Zahlwort' # Wird später wieder gelöscht, dient der Kompatibilität zu lexeme_manuell

In [ ]:
# Substantive ergänzen
mask = zahlwörter_all.lex == zahlwörter_all.lemma
df = zahlwörter_all[mask].copy()
df['lex']   = df.lex.str.capitalize()
df['lemma'] = df.lex.str.capitalize()
df['tag']   = 'NN'
zahlwörter_all = pak.add_rows(zahlwörter_all, df)
#df

In [ ]:
#zahlwörter_all['score'] = wiktionary.score.quantile(0.5)

In [ ]:
# zahlwörter_all anschauen
pak.sample(zahlwörter_all)

## lexeme_manuell

In [ ]:
# laden
lexeme_manuell = pak.load_excel( lexeme_manuell_filename, tabcol='tab' )
lexeme_manuell = pak.move_cols(  lexeme_manuell, 'tag_lex')

In [ ]:
#pak.sample(lexeme_manuell)

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
# Partizipien im falschen Tab definiert?
mask1 = lexeme_manuell.tag.isin(['ADJ1','ADJ2'])
mask2 = lexeme_manuell.tab != 'Partizipien'
mask = mask1  &  mask2
check_mask(lexeme_manuell,mask,0)

In [ ]:
# Partizipien falsch definiert?
mask1 =  lexeme_manuell.tab == 'Partizipien'
mask2 = ~lexeme_manuell.tag.isin(['ADJ1','ADJ2'])
mask = mask1  &  mask2
check_mask(lexeme_manuell,mask,0)

In [ ]:
# Partizipien falsch definiert?
mask1 = lexeme_manuell.tab == 'Partizipien'
mask2 = lexeme_manuell.verb == ''
mask = mask1  &  mask2
check_mask(lexeme_manuell,mask,0)

In [ ]:
# Partizipien falsch definiert?
mask = lexeme_manuell.member.str.contains('Ptz')
check_mask(lexeme_manuell,mask,0)

In [ ]:
# Abkürzungen falsch definiert?
# Das Schlüsselwort 'Abkürzung' darf nirgends in lexeme_manuell vorkommen.
mask1A = lexeme_manuell.member.str.contains('Abkürzung')
mask1B = lexeme_manuell.member.str.contains('Abkürzung')
mask = (mask1A | mask1B) 
check_mask(lexeme_manuell,mask,0)
#lexeme_manuell[mask]

In [ ]:
# score
lexeme_manuell['score'] = wiktionary.score.quantile(0.8)

In [ ]:
#pak.sample(zahlwörter_all)

In [ ]:
s = zahlwörter_all.columns
t = lexeme_manuell.columns
bpy.minivenn(s,t, format='print2')

In [ ]:
# assert subset
assert set(zahlwörter_all.columns) - set(['def']) <= set(lexeme_manuell.columns)

In [ ]:
# zahlwörter_all anfügen
lexeme_manuell = pak.add_rows(lexeme_manuell, zahlwörter_all)

In [ ]:
# tab analysieren
pak.analyse_freqs( lexeme_manuell, cols=['tab','tag'])

In [ ]:
# NaN

# def
lexeme_manuell['def'] = lexeme_manuell['def'].fillna('')

# def
lexeme_manuell['verb'] = lexeme_manuell['verb'].fillna('')

# member
mask = lexeme_manuell.member.isnull()   |   lexeme_manuell.member.isin(['nan'])
lexeme_manuell.loc[mask,'member'] = ''
lexeme_manuell.member += ' manuell'

# lemma und lex
mask = lexeme_manuell.lemma.isnull()   |   lexeme_manuell.lemma.isin(['nan',''])
lexeme_manuell.loc[mask,'lemma'] = lexeme_manuell[mask].lex
#lexeme_manuell.loc[mask,'member'] += ' ergänzt'

mask = lexeme_manuell.lex.isnull()   |   lexeme_manuell.lex.isin(['nan',''])
lexeme_manuell.loc[mask,'lex'] = lexeme_manuell[mask].lemma
#lexeme_manuell.loc[mask,'member'] += ' ergänzt'

# genus
mask = lexeme_manuell.genus.isnull()   |   lexeme_manuell.genus.isin(['nan'])
lexeme_manuell.loc[mask,'genus'] = ''

# tag_lex
mask = lexeme_manuell.tag_lex.isnull()   |   lexeme_manuell.tag_lex.isin(['nan'])
lexeme_manuell.loc[mask,'tag_lex'] = ''

mask = (lexeme_manuell.lex != lexeme_manuell.lemma)   &   (lexeme_manuell.tag_lex == '')
lexeme_manuell.loc[mask,'tag_lex'] = 'lex'

mask = (lexeme_manuell.lex == lexeme_manuell.lemma)   &   (  (lexeme_manuell.tag_lex == 'lex')  | (lexeme_manuell.tag_lex == '')  )
lexeme_manuell.loc[mask,'tag_lex'] = 'lemma'

# tag
mask = lexeme_manuell.tag.isnull()   |   lexeme_manuell.tag.isin(['nan',''])
check_mask( lexeme_manuell, mask, 0, 0)

# Abschluss
lexeme_manuell.member = lexeme_manuell.member.str.strip()
lexeme_manuell = pak.rename_col(lexeme_manuell, 'genus','Genus')

#lexeme_manuell = pak.move_cols(lexeme_manuell, ['Genus','abk'], -1)
#lexeme_manuell['abk'] = ''

In [ ]:
# Dups löschen
mask = lexeme_manuell.duplicated()
lexeme_manuell, df = pak.move_rows(lexeme_manuell,mask)
grid(df)

In [ ]:
pak.sample(lexeme_manuell)

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
# strip
for c in pak.col_names(lexeme_manuell, only='str'):
    lexeme_manuell[c] = lexeme_manuell[c].str.replace('  ',' ').str.strip()

In [ ]:
# member analysieren
pak.analyse_freqs( lexeme_manuell, cols=['member',], splits=[' '])

In [ ]:
# Derzeit nicht nötig: Ersetzungen im member
r = {  'Abkürzungen'        :  'Abkürzung',      
       'Abkürzung Abkürzung':  'Abkürzung',        
    }
lexeme_manuell['member'] = pak.replace_str(lexeme_manuell.member, r)
#grid(lexeme_manuell)

In [ ]:
# Tab Esszett: tag_lex setzen
mask1 = lexeme_manuell.tab == 'esszett'
mask2A = lexeme_manuell.tag_lex == ''
mask2B = lexeme_manuell.tag_lex == 'lex'
mask = mask1  &  (mask2A | mask2B)
check_mask(lexeme_manuell,mask,653)
lexeme_manuell.loc[mask,'tag_lex'] = 'lexAlt'

In [ ]:
# Tab Esszett: member setzen
mask1 =  lexeme_manuell.tab == 'esszett'
mask2 = ~lexeme_manuell.member.str.contains('lexAlt')
mask = mask1  &  mask2
check_mask(lexeme_manuell,mask)
lexeme_manuell.loc[mask,'member'] += 'lexAlt'

In [ ]:
# Tab Abkuerzungen: tag_lex setzen
mask = (lexeme_manuell.tab == 'Abkuerzungen')
lexeme_manuell.loc[mask,'tag_lex'] = 'abk'

mask = lexeme_manuell.member.str.contains('Abkürzung')
lexeme_manuell.loc[mask,'tag_lex'] = 'abk'

In [ ]:
# anschauen
pak.sample(lexeme_manuell)

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
# tag_lex analysieren
r = pak.analyse_freqs(lexeme_manuell, ['tag_lex', 'lex', 'member'], splits=[None,None,' '])
grid(r)

In [ ]:
# Kopie erzeugen, verwenden wir später um alle Partizipien als Adjektive einpflegen zu können
# TODO Nötig??
lexeme_manuell_copy = lexeme_manuell.copy()

In [ ]:
# anschauen
pak.sample(lexeme_manuell)

In [ ]:
# Spalte flexion erstellen und lex darin verpacken
lexeme_manuell['lex_bak'] = lexeme_manuell.lex.copy()
lexeme_manuell = pak.implode_to_defaultdict(lexeme_manuell, col_key='tag_lex', col_value='lex', col_result='flexion')

In [ ]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [ ]:
# lex == lemma: flexion soll leeres defaultdict sein
mask = (lexeme_manuell.lex_bak == lexeme_manuell.lemma) 
lexeme_manuell.loc[mask,'flexion'] = lexeme_manuell[mask].flexion.apply(lambda x: defaultdict(list))  # auf leeres defaultdict setzen
lexeme_manuell = pak.drop_cols(lexeme_manuell, 'lex_bak')
#grid(lexeme_manuell,~mask)

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
# Weitere Spalten in flexion verpacken (mit cols_to_defaultdict
lexeme_manuell = pak.cols_to_defaultdict(  lexeme_manuell, col_dict='flexion', cols_add=['Genus','def']   )  

In [ ]:
# Spalte tab brauchen wir nicht mehr >> löschen
lexeme_manuell = pak.drop_cols(lexeme_manuell, ['tab'])

In [ ]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [ ]:
# gruppieren
lexeme_manuell = pak.group_and_agg(lexeme_manuell,
                               ['lemma', 'tag',   'member',             'verb',   'flexion',             'score', ],
                               ['group', 'group',  pak.agg_words_nodup, 'group',   pak.agg_defaultdicts, 'max',   ],
                               ['lemma', 'tag',   'member',             'verb',   'flexion',             'score', ]
                              )

In [ ]:
# Felder ergänzen
lexeme_manuell['title']       = lexeme_manuell.lemma
lexeme_manuell['lemma_lower'] = lexeme_manuell.lemma.str.lower() 
lexeme_manuell = pak.move_cols( lexeme_manuell, 'lemma_lower','lemma')

In [ ]:
# tagZ und tagZZ ermitteln
lexeme_manuell = pflege_tag_felder(lexeme_manuell, translate_tagZ)

In [ ]:
# anschauen
grid(lexeme_manuell, pak.sample)

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
#suche_debug

In [ ]:
# anschauen: mehr als ein tag
mask = (lexeme_manuell.tag.str.split().str.len() > 1)
lexeme_manuell[mask]

In [ ]:
# section_id2 schreiben
lexeme_manuell['section_id2'] = lexeme_manuell.lemma.str.replace(' ','_') + '_M'
mask = lexeme_manuell.tag == 'CARD'
lexeme_manuell.loc[mask,'section_id2'] = lexeme_manuell[mask].lemma.str.replace(' ','_') + '_Z'
lexeme_manuell = pak.move_cols(lexeme_manuell, ['section_id2','title'])

In [ ]:
# suche_debug
#search_str(lexeme_manuell, suche_debug)

In [ ]:
# Bekannte section_id2 aus wiktionary übertragen

mask = (wiktionary.lemma == wiktionary.title)  # wichtig! Gerade die Datensätze, die nicht darunter fallen, verursachen Probleme bei Partizipien.

# tagZZ
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tagZZ'], col='section_id2',  col_score='score'  )

#tagZ
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tagZ'],  col='section_id2',  col_score='score'  )

# tag
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma_lower','tag'],   col='section_id2',  col_score='score'  )

# Wenn ein sinnvoller tag fehlt, reicht lemma
mask = (wiktionary.lemma == wiktionary.title)   &   wiktionary.tag.str.contains('TODO')
lexeme_manuell = pak.update_col(lexeme_manuell, wiktionary[mask], on=['lemma'], col='section_id2',  col_score='score' )

In [ ]:
# suche_debug
search_str(lexeme_manuell, suche_debug)

In [ ]:
# anschauen
#grid(lexeme_manuell, pak.sample_100)

In [ ]:
# Neue Spalte
wiktionary['verb'] = ''

In [ ]:
bpy.minivenn(wiktionary.columns, lexeme_manuell.columns, format='print2')

In [ ]:
# anfügen
wiktionary = pak.add_rows(wiktionary, lexeme_manuell, assert_subset=True)

In [ ]:
# section_id2 ist jetzt nicht mehr eindeutig
mask = wiktionary.duplicated(subset=['section_id2'], keep=False)
check_mask(wiktionary,mask,5507) 
#a = wiktionary[mask].sort_values(['section_id2'])
#grid(a)

In [ ]:
assert pak.ntypes(wiktionary.flexion) == 1

## ipa, syllables & rhymes

In [ ]:
# Datentypen sind in jeder Zeile gleich
assert pak.ntypes(wiktionary.ipa) == 1
assert pak.ntypes(wiktionary.syllables) == 1

In [ ]:
# Listen zu Strings
mask = ~wiktionary.ipa.isnull()
wiktionary.loc[mask,'ipa'] = wiktionary[mask].ipa.str[0]

mask = ~wiktionary.syllables.isnull()
wiktionary.loc[mask,'syllables'] = wiktionary[mask].syllables.str.join('⇀')

mask = ~wiktionary.rhymes.isnull()
wiktionary.loc[mask,'rhymes'] = wiktionary[mask].rhymes.str[0]

In [ ]:
# cols_to_defaultdict
# 1 min
wiktionary = pak.cols_to_defaultdict( wiktionary, col_dict='flexion', cols_add=['ipa','syllables','rhymes']  )

In [ ]:
# wiktionary anschauen
#grid(wiktionary, pak.sample)

## flexion nach wiktionary_merkmal exploden
* braucht viel RAM
* hiernach muss die löschliste nochmal angewendet werden

In [ ]:
# flexion: vorbereiten
# 5 min
mask1 = wiktionary.flexion.notnull() 
mask2 = wiktionary.flexion != {}
mask3 = wiktionary.flexion != defaultdict(list)
mask = mask1  &  mask2  &  mask3 

spalten = ['section_id2','flexion']
df = pak.explode_dict(wiktionary[spalten][mask], col_dict='flexion', col_key='node_kontext', col_value='data', from_defaultdict=True)
df['merkmal']    = 'flexion'
df['node_debug'] = ''
df['num']        = ''
df['meta']       = ''

In [ ]:
# change_datatype
wiktionary = pak.change_datatype(wiktionary, verbose=False)

In [ ]:
# Beispiel anschauen
mask = df.section_id2 == suche_debug[0]
grid(df,mask)

In [ ]:
bpy.minivenn(wiktionary_merkmal.columns, df.columns, format='print2')

In [ ]:
# an wiktionary_merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df, assert_subset=True)
wiktionary   = pak.drop_cols(wiktionary, ['flexion'])  

In [ ]:
# suche_debug
mask = wiktionary_merkmal.section_id2.isin(suche_debug[0:1])
grid(wiktionary_merkmal[mask].sort_values(['section_id2','merkmal']))

## Pflegen

In [ ]:
# suche_debug 
mask = wiktionary.title.isin(suche_debug)
grid(wiktionary[mask].sort_values(['section_id2',]))

In [ ]:
# pflege_tag_felder (zur Sicherheit noch mal)
wiktionary = pflege_tag_felder(wiktionary, translate_tagZ)

In [ ]:
# LEX-Kennzeichnung sollte eineindeutig sein, außer bei FIX
mask1 =  wiktionary.tag.str.contains('LEX_')   
mask2 =  wiktionary.tag.str.contains(' ')
mask3 = ~wiktionary.tag.str.contains('FIX')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary,mask, 0)

In [ ]:
# inflected >> bool
wiktionary['inflected'] = wiktionary.inflected.astype('bool')

In [ ]:
pak.nnan(wiktionary)

In [ ]:
# score
mask = wiktionary.score.isnull()
check_mask(wiktionary,mask,0)
#wiktionary['score'] = wiktionary.score.fillna(wiktionary.score.quantile(0.95)).astype('float')

In [ ]:
pak.analyse_cols(wiktionary)

In [ ]:
#pak.sample(wiktionary)

In [ ]:
#a = pak.analyse_freqs(wiktionary,['member','tag','lemma'], splits=[' '])
#gridt(a)

In [ ]:
#pak.sample(wiktionary)

## Partizipien
Die Partizipien finden sich<br> 
a) in wiktionary_lemma (als ADJ1 oder ADJ2) 
b) in wiktionary_merkmal (als Partizip-Flexion des Verbs)<br> 

In [ ]:
# Diese Spalten brauchen wir nicht anzuzeigen
without=['pos','tag_0','tag_1','lang','lang_code','error','lemma_lower']

In [ ]:
# manuelle Partizipien anschauen
mask1 =  wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 = wiktionary.member.str.contains('manuell')
mask = mask1  &  mask2
grid(wiktionary,mask,without=without)

In [ ]:
# Vergewissern: Alle Ptz1 sind ADJ1
mask1 = wiktionary.member.str.contains('Ptz1')
mask2 = wiktionary.tag != 'ADJ1'
mask = mask1  &  mask2
check_mask(wiktionary,mask,0)

In [ ]:
# Vergewissern: Alle Ptz2 sind ADJ2
mask1 = wiktionary.member.str.contains('Ptz2')
mask2 = wiktionary.tag != 'ADJ2'
mask = mask1  &  mask2
check_mask(wiktionary,mask,0)

In [ ]:
# Ptz-Markierung ganz aus member entfernen
wiktionary['member'] = pak.remove_words(wiktionary.member, ['Ptz1','Ptz2'] )
mask1 = wiktionary.member.str.contains('Ptz')
check_mask(wiktionary,mask,0)

In [ ]:
# member säubern
r = ['Adjektiv','Wortverbindung']
mask = wiktionary.tag.isin(['ADJ1','ADJ2'])
wiktionary.loc[mask,'member'] = pak.remove_str(wiktionary[mask].member,r)

In [ ]:
# Falsche Partizipien löschen
falsche_partizipien = ['sehen','hören','wollen','lassen','mögen','sollen',]
mask1 = wiktionary.title.isin(falsche_partizipien) 
mask2 =  wiktionary.tag.isin(['ADJ1','ADJ2'])
mask = mask1  &  mask2 
check_mask(wiktionary, mask, 0, 10)

msg = 'Falsche Partizipien'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg,without=without)

In [ ]:
# Verb merken
mask1 =  wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 =  wiktionary.verb == ''  # oder ~wiktionary.member.str.contains('manuell')
mask = mask1  &  mask2
check_mask(wiktionary,mask,27150)

wiktionary.loc[mask,'verb'] = wiktionary[mask].lemma.copy()
grid(wiktionary,mask,without=without)

In [ ]:
# Verb korrigieren
mask1 =  wiktionary.tag == 'ADJ1'
mask2 = ~wiktionary.verb.str.endswith(verbendungen)
mask3 =  wiktionary.verb.str.endswith('end')
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary,mask,10)

wiktionary.loc[mask,'verb'] = wiktionary[mask].lemma.str[:-1]
grid(wiktionary,mask,without=without)

In [ ]:
# lemma = title setzen 
mask =  wiktionary.tag.isin(['ADJ1','ADJ2'])
wiktionary.loc[mask,'lemma'] = wiktionary[mask].title
check_mask(wiktionary,mask,27562)
#grid(wiktionary,mask,without=without)

In [ ]:
# Irreparable Partizipien (werden als lexeme_manuell eingespeist)
mask1 =  wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 = ~wiktionary.verb.str.endswith(verbendungen)
mask = mask1  &  mask2  
check_mask(wiktionary,mask,5)

#wiktionary.loc[mask,'tag']  = 'ADJA' # degradieren
#wiktionary.loc[mask,'verb'] = '' 
#grid(wiktionary, mask)

msg = 'Irreparable Partizipien'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
mask = wiktionary_trash.msg == msg
prüfe_adja = set(wiktionary_trash[mask].lemma)
spalten = wiktionary_trash.columns
grid(wiktionary_trash, msg, without=without)

In [ ]:
# Statt zu löschen hätte man auch einfach normale 'ADJA'-Datensätze draus machen können.
# Hier wird kontrolliert, ob denn für jeden gelöschten DS ein alternativer DS vorhanden ist.

mask1 = wiktionary.tag.isin(['ADJ1','ADJ2','ADJA',])
mask2 = wiktionary.lemma.isin(prüfe_adja)
mask = mask1  &  mask2

gefundene_adja = set(wiktionary[mask].lemma)
assert prüfe_adja == gefundene_adja
#bpy.minivenn(prüfe_adja,gefundene_adja,format='print2')

In [ ]:
# Wenn es verb nicht auch als Wort gibt, dann ist das Müll und kann weg.
# Wenn man das nicht will: Als Partizipien in lexeme_manuell aufnehmen!

mask_Verben = (wiktionary.tagZZ == 'V')  &  (wiktionary.lemma == wiktionary.title)
mask1 = wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 = ~wiktionary.verb.isin(wiktionary[mask_Verben].lemma)
mask = mask1  &  mask2
check_mask(wiktionary,mask,0,10)

msg = 'Verb fehlt'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg, without=without)

In [ ]:
# data_id ermitteln
wiktionary['data_id'] = ''
mask_Verben = (wiktionary.tagZZ == 'V')  &  (wiktionary.lemma == wiktionary.title)

mask = wiktionary.tag.isin(['ADJ1','ADJ2',])
wiktionary.loc[mask] = pak.update_col( df_to   = wiktionary[mask],  
                                       df_from = wiktionary[mask_Verben], 
                                       left_on='verb',       
                                       right_on='lemma', 
                                       col_score='score',
                                       col='section_id2', 
                                       col_rename='data_id'                                 
                                    )

In [ ]:
# Alle Partizipien haben eine data_id abbekommen:
# dann Spalte verb löschen
mask1 = wiktionary.tag.isin(['ADJ1','ADJ2',])
mask2 = wiktionary.data_id == ''
mask = mask1  &  mask2
check_mask(wiktionary,mask,0)

wiktionary = pak.drop_cols(wiktionary, ['verb'] )

In [ ]:
# ergänze_felder nochmal
mask = wiktionary.tag.isin(['ADJ1','ADJ2',])
wiktionary.loc[mask] = ergänze_felder(wiktionary[mask])

In [ ]:
# anschauen
mask1 = wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 = wiktionary.member.str.contains('manuell')
mask = mask1  &  mask2
                                       
grid( wiktionary[mask].sort_values(['tag','title']), pak.sample, without=without)

### IDs zusammenlegen

In [ ]:
search_str(wiktionary,'beeindruckend').sort_values('section_id2')

### als Flexion des Verbs eintragen

In [ ]:
# wiktionary_merkmal_partizip erstellen
spalten = ['data_id','lemma','section_id2','tag']
mask = wiktionary.tag.isin(['ADJ1','ADJ2',])
wiktionary_merkmal_partizip = wiktionary[mask][spalten].copy()

wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'section_id2',   'data_ID')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'data_id',       'section_id2')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'data_ID',       'data_id')
wiktionary_merkmal_partizip = pak.rename_col(wiktionary_merkmal_partizip, 'lemma',         'data')
wiktionary_merkmal_partizip['merkmal']      = 'Ptz' + wiktionary_merkmal_partizip.tag.str[-1]
wiktionary_merkmal_partizip['node_debug']   = '' 
wiktionary_merkmal_partizip['node_kontext'] = '' 
wiktionary_merkmal_partizip['num']          = '' 
wiktionary_merkmal_partizip['meta']         = '' 
wiktionary_merkmal_partizip = pak.drop_cols(wiktionary_merkmal_partizip, 'tag')

In [ ]:
# anschauen
pak.sample(wiktionary_merkmal_partizip)

In [ ]:
# wiktionary_partizip suche_debug
search_str(wiktionary_merkmal_partizip, suche_debug)

In [ ]:
# Spalten vergleichen
bpy.minivenn(wiktionary_merkmal.columns,wiktionary_merkmal_partizip.columns, format='print2')

In [ ]:
# wiktionary_merkmal_partizip als merkmal anfügen
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, wiktionary_merkmal_partizip)

In [ ]:
# Einige als Adjektiv ungeeignete Partizipien löschen
# (als Partizipformen des Verbs waren sie geeignet)
mask1 =  wiktionary.tag.isin(['ADJ1','ADJ2'])
mask2 = ~wiktionary.lemma.str.endswith(('d','t','en'))
mask3 = ~wiktionary.lemma.isin(['vertan','angetan','zugetan'])
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary,mask,36)

msg = 'als Adjektiv ungeeignete Partizipien'
wiktionary, wiktionary_trash = pak.move_rows(df_from=wiktionary, df_to=wiktionary_trash, mask=mask, msg=msg) 
grid(wiktionary_trash, msg)

In [ ]:
raise Stop

## wiktionary in 2 Teile aufsplitten

### wiktionary_nolemma

In [ ]:
wiktionary_nolemma       = wiktionary.head(0)
wiktionary_nolemma_trash = wiktionary.head(0)

In [ ]:
# LEX-Datensätze nach wiktionary_nolemma verschieben
mask = wiktionary.tag.str.contains('LEX_')    
check_mask(wiktionary,mask,639734)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='tag_LEX' )  

In [ ]:
# auch nach wiktionary_nolemma verschieben
mask = wiktionary.member.str.contains('Deklinierte_Form')   # 2 Datensätze
check_mask(wiktionary, mask, 0, 10)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='member_Deklinierte_Form' )  

In [ ]:
# auch nach wiktionary_nolemma verschieben
mask = (wiktionary.title != wiktionary.lemma)               
check_mask(wiktionary, mask, 10)
wiktionary, wiktionary_nolemma = pak.move_rows( df_from=wiktionary, df_to=wiktionary_nolemma, mask=mask, msg='title_ungleich_lemma' )  

In [ ]:
# wiktionary suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

In [ ]:
# wiktionary_nolemma: Spalten pflegen
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'title', 'lex')
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'section_id2','nolemma_id')
wiktionary_nolemma = pak.rename_col(wiktionary_nolemma, 'score', 'data_score')
wiktionary_nolemma = pak.drop_cols(wiktionary_nolemma, ['pos','tag_anz','ipa','syllables','msg','page_id','page_part','inflected','rhymes'])
#wiktionary_nolemma = pak.rename_col( wiktionary_nolemma, 'tag_0', 'tag') # Jetzt gibt es definitiv keine Mehrfachtags mehr

In [ ]:
# anschauen
pak.sample(wiktionary_nolemma)

### wiktionary_lemma

In [ ]:
# wiktionary_lemma: erstellen und Spalten pflegen
wiktionary_lemma  = pak.drop_cols(wiktionary, ['title','pos'])
wiktionary_lemma = pak.rename_col(wiktionary_lemma, 'score', 'lemma_score')

In [ ]:
wiktionary_lemma_trash = wiktionary_lemma.head(0)

In [ ]:
#del wiktionary

In [ ]:
# wiktionary_lemma suche_debug
grid(search_str(wiktionary_lemma, suche_debug))

## wiktionary_lemma weiterbearbeiten

### Grundpflege

In [ ]:
# reset_index
wiktionary_lemma = pak.reset_index(wiktionary_lemma)

In [ ]:
# drop_cols
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,['msg','inflected','error','lang','lang_code'])

In [ ]:
# ergänze_felder
#from x32_Funktionen import *
wiktionary_lemma = ergänze_felder(wiktionary_lemma)

In [ ]:
mask = wiktionary_lemma.lemma_score.isnull()
wiktionary_lemma[mask]

In [ ]:
# NaN
#wiktionary_lemma['lemma_score'] = wiktionary_lemma.lemma_score.fillna(0.45)
assert not pak.any_nan(wiktionary_lemma, without=['page_id','page_part','data_id',])
#pak.nnan(wiktionary_lemma)

## wiktionary_merkmal_text

In [ ]:
# ist schon geladen
# wiktionary_merkmal_text = pak.load_pickle(wiktionary_merkmal_text_filename)
assert not pak.any_nan(wiktionary_merkmal_text)
# pak.nnan(wiktionary_merkmal_text)

### Grundpflege

In [ ]:
# drop_cols
wiktionary_merkmal_text = pak.drop_cols( wiktionary_merkmal_text, ['section_id'])

In [ ]:
# wiktionary_merkmal_text: list_to_string
wiktionary_merkmal_text['num'] = pak.list_to_string(wiktionary_merkmal_text.num)

In [ ]:
# wiktionary_merkmal_text: drop_duplicates
vorher = wiktionary_merkmal_text.shape[0]
wiktionary_merkmal_text = wiktionary_merkmal_text.drop_duplicates()
nachher = wiktionary_merkmal_text.shape[0]
print(vorher-nachher,'Datensätze gelöscht')

In [ ]:
pak.sample(wiktionary_merkmal_text)

In [ ]:
# Grobreinigung 

r = { ' !':'!',  ' ?':'?',  ' ,':',',  ' en ':'en ',  ' e ':'e ',  ' en,':'en,',  ' e,':'e,',  ' es,':'es,',  '  ':' ', }
wiktionary_merkmal_text['data'] = pak.replace_str( wiktionary_merkmal_text.data, r )

r = ['[1]', '[2]', '[3]', '[4]', '[5]', '[6]', '[7]', '[8]', '[9]', '[10]', '[11]', '[12]', '[1a]', '[*]']
wiktionary_merkmal_text['data'] = pak.remove_str( wiktionary_merkmal_text.data, r )

In [ ]:
pak.nnan(wiktionary_lemma)

### Aufzählungen in Einzeldatensätze aufsplitten
* Die merkmale bsp_re und bsp_sp enthalten Aufzählungen. Diese werden in Einzeldatensätze aufgesplittet. 
* Das Feld rank_merkmal erhält die Reihenfolge.

In [ ]:
# Splitten vorbereiten

mask = wiktionary_merkmal_text.merkmal.isin(['bsp_re','bsp_sp'])
r = { '–':'➤',  '—':'➤',  ';':'➤',  '/':'➤',  }
wiktionary_merkmal_text.loc[mask,'data'] = pak.replace_str( wiktionary_merkmal_text[mask].data, r )

grid( wiktionary_merkmal_text, mask )

In [ ]:
# rank_merkmal
wiktionary_merkmal_text = wiktionary_merkmal_text.reset_index() # spalte 'index' erzeugen
wiktionary_merkmal_text = pak.rank( wiktionary_merkmal_text, cols_group=['section_id2','merkmal'], col_score='index', find='min', col_target='rank_merkmal')
wiktionary_merkmal_text = pak.rename_col(wiktionary_merkmal_text,'index','i')
wiktionary_merkmal_text = pak.force_index(wiktionary_merkmal_text,'i')

In [ ]:
# zu splittende Datensätze abtrennen
mask1 = wiktionary_merkmal_text.merkmal.isin(['bsp_re','bsp_sp'])
mask2 = wiktionary_merkmal_text.data.str.contains('➤')
mask  = mask1  &  mask2

wiktionary_merkmal_text, df = pak.move_rows( wiktionary_merkmal_text, mask, msg=None )

In [ ]:
# splitten
df['data'] = df.data.str.split('➤')
df = df.explode('data')

In [ ]:
df.head(5)

In [ ]:
# rank_merkmal2
df = df.reset_index()
df = pak.rank(df, cols_group=['section_id2','merkmal','rank_merkmal'], col_score='i', find='min', col_target='rank_merkmal2')
df = pak.force_index(df,'i')

In [ ]:
bpy.minivenn(wiktionary_merkmal_text.columns, df.columns, format='print2')

In [ ]:
df.head(5)

In [ ]:
# wieder anfügen
wiktionary_merkmal_text = pak.add_rows(wiktionary_merkmal_text, df)
wiktionary_merkmal_text['rank_merkmal2'] = wiktionary_merkmal_text.rank_merkmal2.fillna(0) 

In [ ]:
# rank_merkmal endgültig schreiben
assert wiktionary_merkmal_text.rank_merkmal2.max() < 100
assert not pak.any_nan(wiktionary_merkmal_text.rank_merkmal)
assert not pak.any_nan(wiktionary_merkmal_text.rank_merkmal2)
wiktionary_merkmal_text['rank_merkmal'] = (wiktionary_merkmal_text.rank_merkmal * 100 + wiktionary_merkmal_text.rank_merkmal2).astype('int')
wiktionary_merkmal_text = pak.drop_cols(wiktionary_merkmal_text,['rank_merkmal2'])

In [ ]:
#mask = wiktionary_merkmal_text.merkmal.isin( ['bsp_re'] )
#grid(wiktionary_merkmal_text,mask)

## Preprocessing

In [ ]:
text = '€ ' + bj_nlp.TESTTEXT_0
text

In [ ]:
# Standardkonfiguration
bj_nlp.preprocess(text)

In [ ]:
a = 'Hej U+1F600 världen'
print(a)

In [ ]:
pak.analyse_datatypes(wiktionary_merkmal)

In [ ]:
# wiktionary_lemma
cols = pak.col_names(wiktionary_lemma, only='str')
cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
#print(cols)

for feld in cols:
    print(feld)
    wiktionary_lemma[feld] = wiktionary_lemma[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)       )

In [ ]:
# wiktionary_nolemma
cols = pak.col_names(wiktionary_nolemma, only='str')
cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
#print(cols)

for feld in cols:
    print(feld)
    wiktionary_nolemma[feld] = wiktionary_nolemma[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

In [ ]:
# wiktionary_merkmal

if gründlich:
    cols = pak.col_names(wiktionary_merkmal, only='str')
    cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
    # print(cols)
    
    for feld in cols:
        print(feld)
        wiktionary_merkmal[feld] = wiktionary_merkmal[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

In [ ]:
# Doppelspaces prüfen
if gründlich:
    mask = wiktionary_merkmal.data.str.contains('  ')
    check_mask( wiktionary_merkmal, mask, 0)
    #wiktionary_merkmal[mask]

In [ ]:
# wiktionary_merkmal_text

if gründlich:
    cols = pak.col_names(wiktionary_merkmal_text, only='str')
    cols = [ c for c in cols if c not in ['error']  and not c.startswith('tag')] 
    #print(cols)
    
    for feld in cols:
        print(feld)
        wiktionary_merkmal_text[feld] = wiktionary_merkmal_text[feld].fillna('').apply(   lambda x: bj_nlp.preprocess(x)   )

## Sonderzeichen

In [ ]:
# Sonderfälle
#mask = wiktionary_merkmal.data.str.contains('{{')
r = { '{{NNBSP}}'   :   ' '  ,   
             "''"   :   ''   ,
    }
wiktionary_merkmal['data'] = pak.replace_str(wiktionary_merkmal.data, r)

In [ ]:
# Beispiel anschauen
mask = wiktionary_merkmal.section_id2 == suche_debug[0]
grid(wiktionary_merkmal,mask)

In [ ]:
# wiktionary_lemma enthält nicht_erlaubte_sonderzeichen (siehe Steuertabellen)
mask = wiktionary_lemma.lemma.str.contains(nicht_erlaubte_sonderzeichen_lemma)   &   ~wiktionary_lemma.section_id2.str.endswith('_M')
check_mask(wiktionary_lemma, mask, 0, 50)

msg = 'lemma_nicht_erlaubte_sonderzeichen'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows(df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)
grid(wiktionary_lemma_trash,msg, color='green')

In [ ]:
# wiktionary_nolemma enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_nolemma.lemma.str.contains(nicht_erlaubte_sonderzeichen_lemma) # siehe Steuertabellen
check_mask(wiktionary_nolemma, mask, 0, 100)

msg='lemma_nicht_erlaubte_sonderzeichen'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg)
grid(wiktionary_nolemma_trash,msg, color='green')

In [ ]:
# wiktionary_nolemma enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_nolemma.lex.str.contains(nicht_erlaubte_sonderzeichen_lemma) # siehe Steuertabellen
check_mask(wiktionary_nolemma, mask, 0, 100)

msg='lex_nicht_erlaubte_sonderzeichen'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows(df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg)
grid(wiktionary_nolemma_trash,msg, color='green')

In [ ]:
# data remove_str
r = ['Verzeichnis:Deutsch/','Verzeichnis:','<sub>','</sub>','<sup>','</sup>','Wikipedia:de:','Wikipedia:',]
wiktionary_merkmal['data'] = pak.remove_str(wiktionary_merkmal.data, r )

In [ ]:
# data enthält Reste von Tags >> absplitten
mask = wiktionary_merkmal.data.str.contains('<',regex=False)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.split('<').str[0]
#wiktionary_merkmal[mask]

In [ ]:
# data enthält Reste von Tags >> absplitten
mask = wiktionary_merkmal.data.str.contains('{',regex=False)
wiktionary_merkmal.loc[mask,'data'] = wiktionary_merkmal[mask].data.str.split('{').str[0]
#wiktionary_merkmal[mask]

In [ ]:
# wiktionary_merkmal data belöschen
mask = wiktionary_merkmal.data.str.startswith(('http:','https:'))
check_mask(wiktionary_merkmal, mask, 65)

msg='data_startswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

In [ ]:
# wiktionary_merkmal data belöschen
mask = wiktionary_merkmal.data.str.lower().str.endswith(('.jpg','.png','.gif'))
check_mask(wiktionary_merkmal, mask, 6)

msg='data_endswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

In [ ]:
# wiktionary_merkmal node_kontext belöschen
mask = wiktionary_merkmal.node_kontext.str.startswith(('mini|','thumb|','|'))
check_mask(wiktionary_merkmal, mask, 40)

msg='node_kontext_startswith'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

In [ ]:
# wiktionary_merkmal node_kontext belöschen
mask = wiktionary_merkmal.node_kontext.str.contains('px|', regex=False)
check_mask(wiktionary_merkmal, mask, 3)

msg='node_kontext_contains'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

In [ ]:
# wiktionary_merkmal enthält nicht_erlaubte_sonderzeichen
# Das weist auf Fehler beim Parsen hin >> löschen

felder = ['node_debug','node_kontext','meta','data'] 
for feld in felder: 
    mask = wiktionary_merkmal[feld].str.contains(nicht_erlaubte_sonderzeichen_gnädig) 
    print(feld, wiktionary_merkmal[mask].shape[0], 'Datensätze')
    wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg= feld + '_sonderzeichen', verbose=False)

In [ ]:
# Häufigkeiten
mask = wiktionary_merkmal_trash.msg.str.endswith('_sonderzeichen')
a = pak.analyse_freqs(wiktionary_merkmal_trash[mask], ['msg','merkmal'])
a

In [ ]:
# Anzeigen
#mask = wiktionary_merkmal_trash.msg.str.endswith('_sonderzeichen')
#gridt(wiktionary_merkmal_trash[mask])

In [ ]:
assert not pak.any_nan(wiktionary_merkmal.data) 

In [ ]:
#a = pak.analyse_freqs(wiktionary_merkmal_trash,['msg','data','meta'])
#grid(a)

In [ ]:
#mask = (wiktionary_merkmal_trash.msg == 'data_endswith')
#grid(wiktionary_merkmal_trash,mask)

In [ ]:
# Beispiel anschauen
#mask = wiktionary_merkmal.section_id2 == suche_debug[0]
#grid(wiktionary_merkmal,mask)

## wiktionary_merkmal

### node_debug >> member
* member durch Daten aus der Spalte node_debug ergänzen

In [ ]:
# übersicht reinigen mit remove_str und replace_str
mask = (wiktionary_merkmal.merkmal == 'übersicht')
r = ['-sch', 'Bairisch','Deutsch', ' f',' m',' n', 'Übersicht']
wiktionary_merkmal.loc[mask, 'node_debug'] = pak.remove_str(wiktionary_merkmal[mask].node_debug, r)

r = {'Pronomina-Tabelle': 'Pronomen', 'adjektivisch':'adjNN', 'Eigenname':'Name'}
wiktionary_merkmal.loc[mask, 'node_debug'] = pak.replace_str(wiktionary_merkmal[mask].node_debug, r)

In [ ]:
# Was enthält node_debug?
mask = (wiktionary_merkmal.merkmal == 'übersicht')
a = pak.analyse_freqs(wiktionary_merkmal[mask], ['node_debug', 'section_id2'], splits=[' ',None])
a

In [ ]:
# member_plus festlegen
mask = (wiktionary_merkmal.merkmal == 'übersicht')   &   (wiktionary_merkmal.node_debug.str.len() > 0)
spalten = ['section_id2','node_debug']
df = wiktionary_merkmal[mask][spalten].drop_duplicates()
df.columns = ['section_id2','member_plus']
#grid(df,sample)

In [ ]:
# an member anhängen
wiktionary_lemma = pak.update_col(wiktionary_lemma, df, on='section_id2',col='member_plus')
mask = (wiktionary_lemma.member_plus.str.len() > 0)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.strip() + ' ' + wiktionary_lemma[mask].member_plus.str.strip()
wiktionary_lemma = pak.drop_cols(wiktionary_lemma, 'member_plus')

In [ ]:
# node_debug leeren
mask = (wiktionary_merkmal.merkmal == 'übersicht')
wiktionary_merkmal.loc[mask,'node_debug'] = ''

In [ ]:
# Letzte Löschungen
r = ['Wortart fehlt']
wiktionary_merkmal.node_debug = pak.remove_str( wiktionary_merkmal.node_debug, r )

In [ ]:
# node_debug löschen (müsste jetzt überall leer sein)
mask = (wiktionary_merkmal.node_debug.str.len() > 0)
if version == '_all':
    check_mask(wiktionary_merkmal, mask, 0, 0)
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, 'node_debug')   

### übersicht und flexion
* aus wiktionary_merkmal herausnehmen, pflegen und wieder anfügen

In [ ]:
# Beispiel anschauen
#mask = wiktionary_merkmal.section_id2 == suche_debug[0]
#grid(wiktionary_merkmal,mask)

In [ ]:
# nach flexion auslagern
mask = wiktionary_merkmal.merkmal.isin(['übersicht','flexion']) 
wiktionary_merkmal, flexion = pak.move_rows( wiktionary_merkmal, mask, msg=None ) 
flexion_trash = flexion.head(0)

In [ ]:
mask = flexion.data == '€'
flexion[mask]

In [ ]:
# In wiktionary_merkmal ist node_kontext jetzt überflüssig
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal, 'node_kontext') 

In [ ]:
flexion = pak.rename_col(flexion, 'merkmal',      'quelle')
flexion = pak.rename_col(flexion, 'node_kontext', 'merkmal_lang')
flexion = pak.drop_cols(flexion, ['num','meta','msg','section_id','quelle'])
flexion['merkmal'] = flexion.merkmal_lang.copy()
flexion = pak.move_cols( flexion, ['section_id2', 'merkmal_lang', 'merkmal', 'data'])

In [ ]:
# Dups entfernen
flexion = pak.reset_index(flexion.drop_duplicates())

In [ ]:
# meta schreiben
# dazu lexeme_manuell nochmal aufklappen, um an die lexeme heranzukommen
df = pak.explode_dict(lexeme_manuell, col_dict='flexion', col_key='key', col_value='data', from_defaultdict=True)
mask = df.section_id2 == 'und'
df[mask]

In [ ]:
# meta schreiben
flexion['meta'] = ''
mask = pak.isin( flexion, df, on=['section_id2','data'])
check_mask(flexion,mask)
flexion.loc[mask,'meta'] = 'manuell'

In [ ]:
# abspalten
flexion, df = pak.move_rows(flexion, mask, msg=None)

# auswählen
df = df.drop_duplicates(subset=['section_id2','merkmal'])

# wieder anfügen
flexion = pak.add_rows(flexion, df, assert_subset=True)

In [ ]:
# merkmal reinigen

# 'Ptz1' und 'Ptz2' maskieren
flexion['merkmal'] = pak.replace_str(flexion.merkmal, {'Ptz1':'PtzA', 'Ptz2':'PtzB'})

# merkmal reinigen
r = ['*','1','2','3','4','stark','schwach','gemischt']
flexion['merkmal'] = pak.remove_str(flexion.merkmal, r)

# 'Ptz1' und 'Ptz2' demaskieren
flexion['merkmal'] = pak.replace_str(flexion.merkmal, {'PtzA':'Ptz1', 'PtzB':'Ptz2'})

In [ ]:
# Beispiel anschauen
#mask = flexion.section_id2 == suche_debug[0]
#grid(flexion,mask)

In [ ]:
# translate_first

print(translate_first1)
print(translate_first2)

for quelle, ziel in translate_first1:
    flexion.merkmal = flexion.merkmal.str.replace( quelle, ziel )

for quelle, ziel in translate_first2:
    flexion.merkmal = flexion.merkmal.str.replace( quelle, ziel )

In [ ]:
translate_merkmal

In [ ]:
# translate_merkmal: merkmal reinigen
flexion['merkmal'] = pak.replace_str(flexion.merkmal, translate_merkmal)

In [ ]:
#a = analyse_freqs(flexion,'merkmal','data', split_A=' ', style='top') # 
#grid(a)

In [ ]:
# strip
flexion.merkmal = flexion.merkmal.str.replace('  ',' ').str.strip()

In [ ]:
# Einige Merkmale löschen
r = ['','Weitere Konjugationen','beschreibung','bezug','breite','Jahr','Titel','am','Zugriff','Monat','Tag','Autor','Bild','Flexion','unpersönlich',
     'Kommentar','zugriff','hrsg','Artikel','größe','titel','Ort','spr','Verlag','Seiten','autor','datum','Weitere','kein-s', 'Kein-ens','Datum',
     'EW','ISBN','Originalsprache','TitelErg','dim','kommentar','name','region','sprache','text','Keine weiteren Formen',   
    ]
mask = flexion.merkmal.isin(r)
check_mask(flexion,mask,1500)

msg = 'ungenutzte_Merkmale'
flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg=msg)
grid(flexion_trash,msg)

In [ ]:
# merkmale sortieren
flexion = sort_merkmale(flexion)

In [ ]:
# Müll löschen (obsolet: Das wurde weiter oben schon gelöscht)
mask = flexion.merkmal.str.contains('|', regex=False)
check_mask(flexion,mask,0)
#flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg='Müll1')

In [ ]:
# Müll löschen (obsolet: Das wurde weiter oben schon gelöscht)
mask = flexion.data.str.contains('<', regex=False)
check_mask(flexion,mask,0)
#flexion, flexion_trash = pak.move_rows(df_from=flexion, df_to=flexion_trash, mask=mask, msg='Müll2') 

In [ ]:
# Striche vereinheitlichen
r = {'—':'-', '–':'-'}
flexion.data = pak.replace_str(flexion.data, r)

In [ ]:
#a = pak.analyse_freqs(flexion,['merkmal','data'], splits=[' ', None]) # 
#gridt(a)

In [ ]:
#a = pak.analyse_freqs(flexion,['merkmal','data']) 
#gridt(a)

In [ ]:
#a = pak.analyse_freqs(flexion,['sort','merkmal']) # 
#gridt(a)

In [ ]:
# flexion wieder an wiktionary_merkmal anfügen

df = pak.drop_cols(flexion, ['merkmal_lang','sort'])
df = df.drop_duplicates()
#grid(df)
wiktionary_merkmal = pak.add_rows(wiktionary_merkmal, df, assert_subset=True)

In [ ]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

In [ ]:
# Speicher sparen
if speicher_freigeben:
    del flexion

### wiktionary_merkmal pflegen
* Diverse Pflegeschritte

In [ ]:
# drop_cols
wiktionary_merkmal = pak.drop_cols(wiktionary_merkmal,['rhymes','inflected','lang','lang_code','lemma_lower'])

In [ ]:
mask = wiktionary_merkmal.merkmal == 'rhymes'
pak.sample(wiktionary_merkmal[mask])

In [ ]:
# change_datatype
wiktionary_merkmal = pak.change_datatype(wiktionary_merkmal)

In [ ]:
# Keine nan in data
assert not pak.any_nan(wiktionary_merkmal.data) 

In [ ]:
# herkunft: meta leeren
mask = wiktionary_merkmal.merkmal.isin(['herkunft']) 
wiktionary_merkmal.loc[mask,'meta'] = ''
#grid(wiktionary_merkmal, mask)

In [ ]:
#grid(wiktionary_merkmal_trash)

In [ ]:
# noPl
mask = (wiktionary_merkmal.data.str.len() <= 1)  &   wiktionary_merkmal.merkmal.str.endswith('Pl')   &   (wiktionary_merkmal.merkmal != 'noPl')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noPl¼'
wiktionary_merkmal.loc[mask,'data']    = '1'

In [ ]:
# noSg
mask = (wiktionary_merkmal.data.str.len() <= 1)  &   wiktionary_merkmal.merkmal.str.endswith('Sg')   &   (wiktionary_merkmal.merkmal != 'noSg')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noSg¼'
wiktionary_merkmal.loc[mask,'data']    = '1'

In [ ]:
# mindestens 2 Zeichen deuten auf noSg hin

# finden
mask = (wiktionary_merkmal.merkmal == 'noSg¼')
df = wiktionary_merkmal[mask].groupby('section_id2')['merkmal'].agg('count').reset_index()
mask = (df.merkmal > 1)
markiere = df[mask]['section_id2']

# nach wiktionary_lemma übertragen
mask = wiktionary_lemma.section_id2.isin(markiere)
wiktionary_lemma.loc[mask,'member'] += ' noSg'

In [ ]:
# mindestens 2 Zeichen deuten auf noPl hin

# finden
mask = (wiktionary_merkmal.merkmal == 'noPl¼')
df = wiktionary_merkmal[mask].groupby('section_id2')['merkmal'].agg('count').reset_index()
mask = (df.merkmal > 1)
markiere = df[mask]['section_id2']

# nach wiktionary_lemma übertragen
mask = wiktionary_lemma.section_id2.isin(markiere)
wiktionary_lemma.loc[mask,'member'] += ' noPl'

In [ ]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['noSg¼','noPl¼'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='noSg¼ noPl¼')

In [ ]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['beschreibung 5','bezug 5','breite 5'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='Artefakte')

In [ ]:
# entfernen
mask = wiktionary_merkmal.merkmal.isin(['Weitere Formen'])
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='Weitere Formen')

In [ ]:
#a = pak.analyse_freqs(wiktionary_merkmal,['merkmal','data'], splits=[' ', None]) # 
#grid(a)

In [ ]:
# data enthält nicht_erlaubte_sonderzeichen
mask = wiktionary_merkmal.data.str.contains(nicht_erlaubte_sonderzeichen_gnädig) # siehe Steuertabellen
check_mask(wiktionary_merkmal, mask, 0, 10000)

msg='Sonderzeichen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg, color='green')

In [ ]:
# E-Merkmale 
mask = (plan_merkmal.collect == 'E')
merkmale_E = list(plan_merkmal[mask].merkmal)
merkmale_E 

In [ ]:
# data ist zu kurz 
mask1 =  wiktionary_merkmal.data.str.len() == 1
mask2 = ~wiktionary_merkmal.merkmal.isin( merkmale_E + ['Genus','ipa','rhymes','def','lex','abk'] )
mask3 = ~wiktionary_merkmal.data.isin(['-'])
mask = mask1  &  mask2  &  mask3
check_mask(wiktionary_merkmal, mask, 0, 30)

msg = 'zu kurz'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

In [ ]:
mask = wiktionary_merkmal.data == '€'
wiktionary_merkmal[mask]

In [ ]:
# wiktionary_merkmal belöschen: Wortbildungen
mask = wiktionary_merkmal.data.str.contains('Wortbildungen/')
check_mask(wiktionary_merkmal,mask,201)

msg = 'Wortbildungen'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows( wiktionary_merkmal, wiktionary_merkmal_trash, mask, msg  ) 
grid(wiktionary_merkmal_trash, msg, color='green')

In [ ]:
# meta remove_str replace_str
wiktionary_merkmal['meta'] = pak.remove_str(  wiktionary_merkmal.meta, '()'         )
wiktionary_merkmal['meta'] = pak.replace_str( wiktionary_merkmal.meta, {'/':' '}    )

In [ ]:
# data Klammer am Ende schließen
mask1 =  wiktionary_merkmal.data.str.contains('(', regex=False)
mask2 = ~wiktionary_merkmal.data.str.contains(')', regex=False)
mask = mask1  &  mask2
wiktionary_merkmal.loc[mask,'data'] += ')'
# grid(wiktionary_merkmal, mask)

In [ ]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

### Kontrolle wiktionary_merkmal
* Diverse Kontrollen

In [ ]:
# plan_merkmal Steuertabelle neu erstellen
try:
    del plan_merkmal
except:
    pass

from Steuertabellen.s01_Steuertabellen import *
plan_merkmal = plan_merkmal_erstellen()
#pak.sample(plan_merkmal)
grid(plan_merkmal)

In [ ]:
# Merkmale lt. Steuertabelle und lt. Realität
mask = (plan_merkmal.plan != 2)
merkmale_plan = set(plan_merkmal[mask].merkmal)
merkmale_data = set(wiktionary_merkmal.merkmal.unique())

In [ ]:
# left_only:  Merkmale, die in er Realität nicht auftreten
# right_only: Merkmale, die nicht geplant sind
bpy.minivenn(merkmale_plan,merkmale_data,format='print2')

In [ ]:
# Fast alle in der Steuertabelle plan_merkmal erfassten Merkmale kommen in wiktionary_merkmal tatsächlich vor.
ausnahmen = {'PTKVZ', 'Konj2', 'Konj', 'lemma', 'down_part', 'Konj1', 'Imp', 'Gerundivum', 'übersicht'}
fehlende_merkmale = bpy.minivenn(merkmale_plan - ausnahmen,merkmale_data,format='dict')['left_only']
assert len(fehlende_merkmale) == 0

In [ ]:
# Tauchen die Ausnahmen in wiktionary_merkmal_trash auf?
# Sollten sie nicht.
mask1 =  wiktionary_merkmal_trash.merkmal.isin(  ausnahmen - {'nurWortart'}  )
mask2 = ~wiktionary_merkmal_trash.merkmal.isin(['übersicht'])
mask = mask1  & mask2
check_mask(wiktionary_merkmal_trash,mask,0)
#wiktionary_merkmal_trash[mask]

In [ ]:
# alle Merkmale in wiktionary_merkmal sind in der Steuertabelle plan_merkmal erfasst
alle_tags = set(translate_tagZ.tag) # tags werden als merkmal akzeptiert
unregistrierte_merkmale = bpy.minivenn(merkmale_plan, merkmale_data - alle_tags, format='dict')['right_only']
print(unregistrierte_merkmale)

In [ ]:
# Probleme anschauen
mask = wiktionary_merkmal.merkmal.isin(unregistrierte_merkmale)
check_mask(wiktionary_merkmal,mask,0,3)
grid(wiktionary_merkmal, mask, color='green')

In [ ]:
# Keine nan in data
assert pak.nnan(wiktionary_merkmal.data) == 0

In [ ]:
# Eindeutige Datentypen
for col in wiktionary_merkmal.columns:
    print(col)
    if col == 'num':
        continue
    assert pak.ntypes(wiktionary_merkmal[col]) == 1

In [ ]:
wiktionary_merkmal.shape

## wiktionary_lemma

In [ ]:
# suche_debug
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

### member erweitern
* Einige Merkmale aus wiktionary_merkmal nach member in wiktionary_lemma verschieben
* und zwar die E-Merkmale. das sind die, bei denen es einzig auf die Existenz ankommt. Diese Merkmale können keine verschiedenen Werte annehmen.

In [ ]:
# noSing >> noSg
mask = (wiktionary_merkmal.merkmal == 'noSing')
wiktionary_merkmal.loc[mask,'merkmal'] = 'noSg'

In [ ]:
# E-Merkmale 
print(merkmale_E)

In [ ]:
# anschauen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
a = pak.analyse_freqs(wiktionary_merkmal[mask],['merkmal','data']) # 
grid(a)

In [ ]:
# update-Tabelle erstellen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
df = pak.group_and_agg(wiktionary_merkmal[mask], 
              col_origins=['section_id2', 'merkmal',             'merkmal',       ],
              col_funcs  =['group',       pak.agg_words_nodup,   'count',         ],
              col_names  =['',            'member_plus',         'count',         ],  
             )
#df

In [ ]:
# member_plus schreiben
wiktionary_lemma = pak.update_col(wiktionary_lemma, df, on='section_id2', col='member_plus')

In [ ]:
# member schreiben
mask = (wiktionary_lemma.member_plus.str.len() > 0)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member + ' ' + wiktionary_lemma[mask].member_plus
wiktionary_lemma = pak.drop_cols(wiktionary_lemma,'member_plus')
#grid(wiktionary_lemma, mask)

In [ ]:
# wiktionary_merkmal belöschen
mask = wiktionary_merkmal.merkmal.isin(merkmale_E)
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg='merkmale_E')

### member pflegen

In [ ]:
# werte_anz
df = pak.group_and_agg(wiktionary_merkmal, 
              col_origins=['merkmal',    'data',                  ],
              col_funcs  =['group',       pak.nunique,            ],
              col_names  =['',           'werte_anz',             ],              
             )
#df

In [ ]:
# merkmale mit kleiner werte_anz
mask = (df.werte_anz < 3)   &   ~df.merkmal.str.startswith(('Nom','Gen','Dat','Akk', 'Konj', 'Ptz1'))
m = list(df[mask].merkmal)

mask = wiktionary_merkmal.merkmal.isin(m)
assert wiktionary_merkmal[mask].shape[0] <= 10  

a = pak.analyse_freqs(wiktionary_merkmal[mask], ['merkmal','data'])
a 

#wiktionary_merkmal[mask]

In [ ]:
wiktionary_merkmal[mask]

In [ ]:
# In list wandeln
wiktionary_lemma.member = wiktionary_lemma.member.str.split()

# nach Priorität sortieren (laut Steuertabellen)
wiktionary_lemma.member = member_sortieren(wiktionary_lemma.member) 

# In string wandeln
wiktionary_lemma.member = wiktionary_lemma.member.str.join(' ')

In [ ]:
# Seltene member finden
member_selten = pak.analyse_freqs(wiktionary_lemma, ['member', 'section_id2'], splits=[' ',None])
mask = (member_selten.member_count == 1)
member_selten = member_selten[mask]
member_selten_member      = list(member_selten.member)
member_selten_section_id2 = list(member_selten.section_id2)

# und entfernen
mask = wiktionary_lemma.section_id2.isin(member_selten_section_id2)
wiktionary_lemma.loc[mask,'member'] = pak.remove_str(wiktionary_lemma[mask].member, member_selten_member)
#wiktionary_lemma[mask]

In [ ]:
member_selten

In [ ]:
# anschauen wiktionary_lemma
a = pak.analyse_freqs(wiktionary_lemma,['member','tag'], splits=[' ', None])
gridt(a)

In [ ]:
# anschauen wiktionary_nolemma
b = pak.analyse_freqs(wiktionary_nolemma,['member','tag'], splits=[' ', None])
gridt(b)

In [ ]:
# suche_debug
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

### member weiter pflegen

In [ ]:
# Ersetzungen im member laut member_replaces (definiert in Steuertabellen)
print(member_replaces)
wiktionary_lemma.loc[:,'member'] = pak.replace_str(wiktionary_lemma.member, member_replaces)

In [ ]:
# redundantes Substantiv
mask = wiktionary_lemma.member.str.contains('Substantiv')   &   (wiktionary_lemma.tagZZ == 'N') 
check_mask( wiktionary_lemma, mask, 104832 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Substantiv','')

In [ ]:
# redundantes Adjektiv
mask = wiktionary_lemma.member.str.contains('Adjektiv')   &   (wiktionary_lemma.tagZ == 'ADJ') 
check_mask( wiktionary_lemma, mask, 14312 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Adjektiv','')

In [ ]:
# redundantes Verb
mask = wiktionary_lemma.member.str.contains('Verb')   &   (wiktionary_lemma.tagZZ == 'V') 
check_mask( wiktionary_lemma, mask, 14333 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Verb','')

In [ ]:
# redundantes istName, außerdem 1 falsches istName das NN ist
mask = wiktionary_lemma.member.str.contains('istName')   &   ( wiktionary_lemma.tagZZ == 'N' ) 
check_mask( wiktionary_lemma, mask, 2339 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('istName','')

In [ ]:
# redundantes Name
mask = wiktionary_lemma.member.str.contains('Name')   &   ( wiktionary_lemma.tag_0.isin(['NE']) | wiktionary_lemma.tag_1.isin(['NE']) )
check_mask( wiktionary_lemma, mask, 177 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Name','')

In [ ]:
# redundantes Konjunktion
#mask = wiktionary_lemma.member.str.contains('Konjunktion')   &   ~wiktionary_lemma.member.str.contains('Konjunktionaladverb')   &   \
#     ( wiktionary_lemma.tag_0.isin(['KOUS','KON','KOKOM','KOUI']) | wiktionary_lemma.tag_1.isin(['KOUS','KON','KOKOM','KOUI']) ) 

mask1 =   wiktionary_lemma.member.str.contains('Konjunktion') 
mask2 =  ~wiktionary_lemma.member.str.contains('Konjunktionaladverb') 
mask3A =  wiktionary_lemma.tag_0.isin(['KOUS','KON','KOKOM','KOUI'])
mask3B =  wiktionary_lemma.tag_1.isin(['KOUS','KON','KOKOM','KOUI'])
mask = mask1  &  mask2  &  (mask3A | mask3B)
check_mask( wiktionary_lemma, mask, 104 )

##wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Konjunktion','')
#grid(wiktionary_lemma[mask])

In [ ]:
# redundantes Indefinitpronomen
mask = wiktionary_lemma.member.str.contains('Indefinitpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PIAT','PIS']) | wiktionary_lemma.tag_1.isin(['PIAT','PIS']) )
check_mask( wiktionary_lemma, mask, 66 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Indefinitpronomen','')

In [ ]:
# redundantes Interjektion
mask = wiktionary_lemma.member.str.contains('Interjektion')   &   ( wiktionary_lemma.tag_0.isin(['ITJ']) | wiktionary_lemma.tag_1.isin(['ITJ']) )
check_mask( wiktionary_lemma, mask, 245 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interjektion','')

In [ ]:
# redundantes Adverb
mask = wiktionary_lemma.member.str.contains('Adverb')   &   ( wiktionary_lemma.tag_0.isin(['ADV']) | wiktionary_lemma.tag_1.isin(['ADV']) )
check_mask( wiktionary_lemma, mask, 1524 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Adverb','')

In [ ]:
# redundantes Subjunktion
mask = wiktionary_lemma.member.str.contains('Subjunktion')   &   ( wiktionary_lemma.tag_0.isin(['KOUS']) | wiktionary_lemma.tag_1.isin(['KOUS']) )
check_mask( wiktionary_lemma, mask, 58 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Subjunktion','')

In [ ]:
# redundantes Personalpronomen
mask = wiktionary_lemma.member.str.contains('Personalpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PPER']) | wiktionary_lemma.tag_1.isin(['PPER']) )
check_mask( wiktionary_lemma, mask, 33 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Personalpronomen','')

In [ ]:
# redundantes Demonstrativpronomen
mask = wiktionary_lemma.member.str.contains('Demonstrativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PDAT']) | wiktionary_lemma.tag_1.isin(['PDAT']) )
check_mask( wiktionary_lemma, mask, 32 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Demonstrativpronomen','')

In [ ]:
# redundantes Artikel
mask = wiktionary_lemma.member.str.contains('Artikel')   &   ( wiktionary_lemma.tag_0.isin(['ART']) | wiktionary_lemma.tag_1.isin(['ART']) )
check_mask( wiktionary_lemma, mask, 16 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Artikel','')

In [ ]:
# redundantes Postposition
mask = wiktionary_lemma.member.str.contains('Postposition')   &   ( wiktionary_lemma.tag_0.isin(['APPO']) | wiktionary_lemma.tag_1.isin(['APPO']) )
check_mask( wiktionary_lemma, mask, 13 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Postposition','')

In [ ]:
# redundantes Relativpronomen
mask = wiktionary_lemma.member.str.contains('Relativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PRELAT']) | wiktionary_lemma.tag_1.isin(['PRELAT']) )
check_mask( wiktionary_lemma, mask, 10 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Relativpronomen','')

In [ ]:
# redundantes Possessivpronomen
mask = wiktionary_lemma.member.str.contains('Possessivpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PPOSS']) | wiktionary_lemma.tag_1.isin(['PPOSS']) )
check_mask( wiktionary_lemma, mask, 9 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Possessivpronomen','')

In [ ]:
# redundantes Negationspartikel
mask = wiktionary_lemma.member.str.contains('Negationspartikel')   &   ( wiktionary_lemma.tag_0.isin(['PTKNEG']) | wiktionary_lemma.tag_1.isin(['PTKNEG']) )
check_mask( wiktionary_lemma, mask, 4 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Negationspartikel','')

In [ ]:
# redundantes Interrogativpronomen
mask = wiktionary_lemma.member.str.contains('Interrogativpronomen')   &   ( wiktionary_lemma.tag_0.isin(['PWAT']) | wiktionary_lemma.tag_1.isin(['PWAT']) )
check_mask( wiktionary_lemma, mask, 4 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interrogativpronomen','')

In [ ]:
# falsches noPl (das betrifft vorwiegend NE-Datensätze)
mask = wiktionary_lemma.member.str.contains('noPl')   &   ~wiktionary_lemma.tag_0.isin(['NN','TODO',])   &   ~wiktionary_lemma.tagZZ.isin(['P'])
check_mask(wiktionary_lemma,mask, 1452)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noPl','')
#grid( wiktionary_lemma, mask )

In [ ]:
# falsches noSg
mask = wiktionary_lemma.member.str.contains('noSg')   &   ~wiktionary_lemma.tag_0.isin(['NN','NE','TODO','PPER'])
check_mask(wiktionary_lemma,mask,3)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noSg','')
#grid( wiktionary_lemma, mask )

In [ ]:
# falsches noSteig
mask = wiktionary_lemma.member.str.contains('noSteig')   &   ~wiktionary_lemma.tag_0.isin(['ADJA'])
check_mask(wiktionary_lemma,mask, 22)
wiktionary_lemma.loc[mask,'member'] = wiktionary_lemma[mask].member.str.replace('noSteig','')

In [ ]:
# falsches noFlex
mask = wiktionary_lemma.member.str.contains('noFlex')   &   ~wiktionary_lemma.tagZ.isin(['ADJ',])
check_mask( wiktionary_lemma, mask, 2 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('noFlex','')

In [ ]:
# falsches Interjektion
mask = wiktionary_lemma.member.str.contains('Interjektion')  &   wiktionary_lemma.tag_0.isin(['PTKANT',])
check_mask( wiktionary_lemma, mask, 31 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Interjektion','')

In [ ]:
# falsches Präfix
mask = wiktionary_lemma.member.str.contains('Präfix')   &   ~wiktionary_lemma.tagZ.isin(['FIX',])
check_mask( wiktionary_lemma, mask, 3 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Präfix','')

In [ ]:
# Taxonomie >> Spezies
mask = wiktionary_lemma.member.str.contains('Taxonomie')   &   (wiktionary_lemma.tagZZ == 'N') 
check_mask( wiktionary_lemma, mask, 169 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Taxonomie','Spezies')

In [ ]:
# falsches Vorname
mask = wiktionary_lemma.member.str.contains('Person_vorname')   &   ~wiktionary_lemma.tag_0.isin(['NE'])
check_mask( wiktionary_lemma, mask, 1 )
wiktionary_lemma.loc[ mask, 'member' ] = wiktionary_lemma[mask].member.str.replace('Person_vorname','')
#grid( wiktionary_lemma, mask )

In [ ]:
# Streichungen im member       
member_removes = [ 'Numerale', 'Pronomen', 'noWortart', 'Hilfsverb', 'Taxonomie', 'Geflügeltes_Wort', ] 

wiktionary_lemma.loc[:,'member'] = pak.remove_str(wiktionary_lemma.member, member_removes)

In [ ]:
# wiktionary_lemma_bak = wiktionary_lemma.copy()
#wiktionary_lemma = wiktionary_lemma_bak.copy()

In [ ]:
# Substantiv aber kein NN
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 = ~wiktionary_lemma.tag.str.contains(' ')
mask = mask1  &  mask2
check_mask(wiktionary_lemma,mask,3)
wiktionary_lemma.loc[mask,'tag'] = 'NN ' + wiktionary_lemma[mask].tag
wiktionary_lemma[mask]

In [ ]:
# korrigieren
wiktionary_lemma, df = pak.move_rows(wiktionary_lemma,mask,msg=None)
df = pflege_tag_felder(df, translate_tagZ)
wiktionary_lemma = pak.add_rows(wiktionary_lemma,df)

In [ ]:
# anschauen
mask1 =  wiktionary_lemma.member.str.contains('Substantiv')
mask2 =  wiktionary_lemma.tag.str.contains(' ')
mask = mask1  &  mask2
pak.analyse_freqs(wiktionary_lemma[mask],['tag','lemma'])

### member prüfen

In [ ]:
# alle member in den realen Daten zusammensammeln

a = pak.analyse_freqs(wiktionary_lemma, ['member','tag'], splits=[' ', None] )
#grid(a)

b = pak.analyse_freqs(wiktionary_nolemma, ['member','tag'], splits=[' ', None] )
#grid(b)

# Member lt. Steuertabelle und lt. Realität
member_plan = set(priorität_member) #- set(member_replaces.keys())

member_data1 = set(a.member)
member_data2 = set(b.member)
member_data  = member_data1.union(member_data2)
#member_data

In [ ]:
# left_only:  member, die geplant sind aber in der Realität nicht vorkommen
# right_only: member, die in der Realität vorkommen aber nicht geplant sind
bpy.minivenn( member_plan, member_data, format='print2')

In [ ]:
# alle Merkmale in wiktionary_merkmal sollten in der Steuertabelle plan_merkmal erfasst sein.
ungeplante_member = bpy.minivenn(member_plan,member_data)['right_only'] - {''}
assert len(ungeplante_member) == 0

In [ ]:
# Beispiel anzeigen 
mask = (wiktionary_merkmal.section_id2 == suche_debug[0])
grid(wiktionary_merkmal,mask)

### wiktionary_lemma pflegen und prüfen
* Diverse Pflegeschritte

In [ ]:
# lemma_lower, isupper, lemma_len
wiktionary_lemma = ergänze_felder(wiktionary_lemma)

In [ ]:
# Eindeutige Datentypen
for col in wiktionary_lemma.columns:
    print(col)
    if col in ['num','error']:
        continue
    assert pak.ntypes(wiktionary_lemma[col]) == 1

In [ ]:
# Keine unbekannten tags
a = pak.analyse_freqs(wiktionary_lemma, ['tag'], splits=[' '])
mask = ~a.tag.isin(alle_tags)
check_mask( a, mask, 0)
a[mask]

In [ ]:
# finden
mask = wiktionary_lemma.tag == 'Zirkumfix'
grid(wiktionary_lemma, mask, color='green')

In [ ]:
# NaN
wiktionary_lemma['page_id']   = wiktionary_lemma.page_id.fillna(-1) 
wiktionary_lemma['page_part'] = wiktionary_lemma.page_part.fillna(-1) 

## Letzte Pflegeschritte

In [ ]:
wiktionary_lemma        = pak.reset_index( wiktionary_lemma.sort_values('section_id2') )

In [ ]:
wiktionary_nolemma      = pak.drop_cols(wiktionary_nolemma,['lang','lang_code','error'])
wiktionary_nolemma      = pak.move_cols(wiktionary_nolemma,'lemma_lower','lemma')

In [ ]:
#wiktionary_merkmal_bak = wiktionary_merkmal.copy()

In [ ]:
# score übertragen und ggf. ergänzen
wiktionary_merkmal = pak.update_col(df_to=wiktionary_merkmal, df_from=wiktionary_lemma,        on='section_id2',                        col='lemma_score', col_rename='data_score', func='max', cond='null')
wiktionary_merkmal = pak.update_col(df_to=wiktionary_merkmal, df_from=wiktionary_nolemma, left_on='section_id2', right_on='nolemma_id', col='data_score',                           func='max', cond='null')
wiktionary_merkmal['data_score'] = wiktionary_merkmal.data_score.fillna(wiktionary_merkmal.data_score.quantile(0.5)).astype('float')

In [ ]:
# Keine Leerzeichen in der ID
mask = wiktionary_lemma.section_id2.str.contains(' ')
check_mask(wiktionary_lemma,mask,0)

mask = wiktionary_nolemma.nolemma_id.str.contains(' ')
check_mask(wiktionary_nolemma,mask,0)

mask = wiktionary_merkmal.section_id2.str.contains(' ')
check_mask(wiktionary_merkmal,mask,0)

## Löschliste nochmal

In [ ]:
# wiktionary_lemma belöschen nach section_id2
mask = wiktionary_lemma.section_id2.isin(löschliste_section_id2)   

msg='löschliste_B1'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)       # Löschen
grid(wiktionary_lemma_trash,msg)

In [ ]:
# wiktionary_lemma belöschen nach lex
mask = wiktionary_lemma.lemma.isin(löschliste_lex)

msg='löschliste_B2'
wiktionary_lemma, wiktionary_lemma_trash = pak.move_rows( df_from=wiktionary_lemma, df_to=wiktionary_lemma_trash, mask=mask, msg=msg)       # Löschen
grid(wiktionary_lemma_trash,msg)

In [ ]:
# wiktionary_nolemma belöschen nach lex
mask = wiktionary_nolemma.lemma.isin(löschliste_lex)   

msg='löschliste_B3'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows( df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg )
grid(wiktionary_nolemma_trash,msg)

In [ ]:
# wiktionary_nolemma belöschen nach lex
mask = wiktionary_nolemma.lex.isin(löschliste_lex)   

msg='löschliste_B4'
wiktionary_nolemma, wiktionary_nolemma_trash = pak.move_rows( df_from=wiktionary_nolemma, df_to=wiktionary_nolemma_trash, mask=mask, msg=msg )
grid(wiktionary_nolemma_trash,msg)

In [ ]:
# wiktionary_merkmal belöschen nach section_id2
mask = wiktionary_merkmal.section_id2.isin(löschliste_section_id2) 

msg='löschliste_B5'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# wiktionary_merkmal belöschen nach lex
mask = wiktionary_merkmal.data.isin(löschliste_lex) 

msg='löschliste_B6'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

In [ ]:
# wiktionary_merkmal belöschen nach data
mask = wiktionary_merkmal.data.isin(löschliste_data) 

msg='löschliste_B7'
wiktionary_merkmal, wiktionary_merkmal_trash = pak.move_rows(df_from=wiktionary_merkmal, df_to=wiktionary_merkmal_trash, mask=mask, msg=msg)
grid(wiktionary_merkmal_trash,msg)

## Prüfen

In [ ]:
# lex_test, lemma_test
from x32_Funktionen import *
lex_test, lemma_test = lade_lex_text( lex_test_filename, level=1 )
#grid(lemma_test)

In [ ]:
# lemma_test
from x32_Funktionen import *
lemma_test = check_lemma_test(lemma_test, wiktionary_lemma)
#lemma_test

In [ ]:
# Fehler
maskA = lemma_test.lemma_id == ''
maskB = lemma_test.check_tag == ''
mask = maskA | maskB
check_mask(lemma_test,mask)

probleme = list(lemma_test[mask].lemma)
lemma_test[mask]

In [ ]:
# Sicherstellen: Keine Fehler
check_mask(lemma_test,mask,0)

## Speichern

In [ ]:
# Doppelspaces prüfen
if gründlich:
    mask = wiktionary_merkmal.data.str.contains('  ')
    check_mask( wiktionary_merkmal, mask, 0)

In [ ]:
pak.analyse_cols(wiktionary_merkmal)

In [ ]:
# wiktionary_merkmal: drop_duplicates
wiktionary_merkmal = wiktionary_merkmal.drop_duplicates()

In [ ]:
assert gründlich

In [ ]:
# Speichern  
pak.dump_pickle(wiktionary_lemma,            wiktionary_lemma_filename_save)   
pak.dump_pickle(wiktionary_nolemma,          wiktionary_nolemma_filename_save)  
pak.dump_pickle(wiktionary_merkmal,          wiktionary_merkmal_filename_save) 
pak.dump_pickle(wiktionary_merkmal_text,     wiktionary_merkmal_text_filename_save)  
os.utime(verzeichnis_save)

In [ ]:
# Save in Google Drive
if environment['in_colab']:
    from google.colab import drive
    drive.mount('/content/drive') 
    !cd /content
    !cp -av {verzeichnis_save} /content/drive/MyDrive

In [ ]:
pak.memory_consumption(locals())

## Trash anschauen

### wiktionary_trash

In [ ]:
wiktionary_trash.msg = wiktionary_trash.msg.fillna('')

In [ ]:
# Übersicht
a = pak.analyse_freqs(wiktionary_trash, ['msg','member'], splits=[None,' '])
a

In [ ]:
# Datensätze
mask = (wiktionary_trash.msg == 'MFIX')
#grid( wiktionary_trash, mask )

In [ ]:
# suche_debug
grid(search_str(wiktionary_trash, suche_debug))

### wiktionary_merkmal_trash

In [ ]:
wiktionary_merkmal_trash.msg = wiktionary_merkmal_trash.msg.fillna('')

In [ ]:
# Übersicht
#a = pak.analyse_freqs(wiktionary_merkmal_trash, ['msg','merkmal', 'data', ])
a = pak.analyse_freqs(wiktionary_merkmal_trash, ['msg','merkmal', 'data', ])
a = pak.drop_cols(a, ['msg_percent','data_count'])
grid(a,  max_column_width=300)

In [ ]:
# Datensätze
mask = (wiktionary_merkmal_trash.msg == 'data_endswith')
grid( wiktionary_merkmal_trash)

In [ ]:
# suche_debug
maskA = wiktionary_merkmal_trash.data.isin(suche_debug)
maskB = wiktionary_merkmal_trash.section_id2.isin(suche_debug)
mask  = maskA | maskB
grid(wiktionary_merkmal_trash,mask, color='green')

### flexion_trash
* flexion_trash enthält Datensätze, die ursprünglich aus wiktionary_merkmal stammen

In [ ]:
# Übersicht
a = pak.analyse_freqs(flexion_trash, ['merkmal', 'data' , 'msg'])
a = pak.drop_cols(a, ['merkmal_percent','msg_count','data_count'])
grid(a)

### wiktionary_lemma_trash

In [ ]:
# suche_debug
mask = wiktionary_lemma_trash.lemma.isin(suche_debug) 
wiktionary_lemma_trash[mask]

In [ ]:
# Übersicht
a = pak.analyse_freqs(wiktionary_lemma_trash,['member','msg','lemma'], splits=[None,' '])
grid(a)

### wiktionary_nolemma_trash

In [ ]:
# suche_debug
mask = wiktionary_nolemma_trash.lemma.isin(suche_debug) 
wiktionary_nolemma_trash[mask]

In [ ]:
# Übersicht
a = pak.analyse_freqs(wiktionary_nolemma_trash,['member','lex'], splits=[None,' '])
grid(a)

## Nutzdaten anschauen

In [ ]:
suche_debug

### wiktionary_lemma

In [ ]:
mask = wiktionary_lemma.lemma.isin(suche_debug) #== 'gefriergetrocknet'
wiktionary_lemma[mask]

### wiktionary_merkmal

In [ ]:
mask = wiktionary_merkmal.section_id2 == suche_debug[0]
a = wiktionary_merkmal[mask].sort_values(['merkmal','section_id2','data'])
grid(a)

In [ ]:
mask = wiktionary_merkmal.data.isin(suche_debug) #== 'gefriergetrocknet'
a = wiktionary_merkmal[mask].sort_values(['merkmal','section_id2','data'])
grid(a)

### wiktionary_nolemma

In [ ]:
# wiktionary_nolemma suche_debug
grid(search_str(wiktionary_nolemma, suche_debug))

In [ ]:
raise Stop